# National Model 5

- Equifax, TU, Experian National model 
- Features selection on PenFed data + Org data model - clientmodel3a
- Updated time split on Equifax
    - Training: 2016-04-01 to 2020-12-31
    - Valid: 2021-01-01 to 2021-03-31

The notebook covers all three steps for creatign a new national model; feature selection, validation model and fold-in model. depending on the applied use case, not each step may be necessary (e.g., if the feature selection does not change compaed to the previous interation, step 1 may be skipped, etc.

In [1]:
import pandas as pd
import numpy as np

import model_engine
model_engine.__version__

'v1.13.0'

In [2]:
import zaml

zaml.__version__

'34.4.1'

In [8]:
from model_engine.power.post_sale import NationalModelBuilder
from model_engine.io.loaders import load_json

import json

In [10]:
import os
import numba as nb
n_jobs = os.getenv('CPU_LIMIT', default=-1)
n_jobs = 20
print(n_jobs)
nb.set_num_threads(n_jobs)

20


In [11]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [12]:
input_client_name = 'penfed'
input_project_name = 'autoloan'
input_bureau_name = 'normalized'
model_type = 'standard_model_member_data' # options are 'standard_model', 'mega_model', 'standard_model_member_data', 'standard_model_ln_data', ''- None defaults to 'standard_model'

print(input_client_name, input_project_name, input_bureau_name, model_type)


penfed autoloan normalized standard_model_member_data


## Create a national model given pre-selected features

In [27]:
# client' feature selection model

feature_selection_model_id = 'clientmodel3a'

In [15]:
feature_selection_model_output_path = f'/d/shared/silver_projects_v2/penfed/autoindirectv1/modeling/{feature_selection_model_id}'
feature_selection_model_output_path

'/d/shared/silver_projects_v2/penfed/autoindirectv1/modeling/clientmodel3a'

In [17]:
ls /d/shared/silver_projects_v2/penfed/autoindirectv1/modeling/clientmodel3a

artifact_manifest.json                  test_scores.parquet
asset.json                              test_target.parquet
best_model_params.json                  test_zest_scores.parquet
calibration_object.obj                  top_features.parquet
feature_definition.parquet              train_app.parquet
feature_importance.parquet              train_auc.json
fit_time_info.json                      train_data_summary.json
input_configuration.json                train_fe_data.parquet
keep_features.json                      train_history.json
key_factors_mapping.json                train_ks.json
model.obj                               train_scores.parquet
model_strategy.json                     train_target.parquet
mrm_pipeline.obj                        train_zest_scores.parquet
parsed_monotonic_constraints_list.json  valid_app.parquet
pipeline.obj                            valid_auc.json
score_recalibration_mapping.json        valid_data_summary.json
splitter.obj                         

In [7]:
keep_features = json.load(open(os.path.join(feature_selection_model_output_path, 'keep_features.json')))
keep_features.pop('client_data')

['client_data_LTV_NORM']

In [8]:
keep_features['collec']

['collec_blnc__sum',
 'collec_months_since_collection_opened__min',
 'collec___count',
 'collec_ratio_paid__max',
 'collec_ratio_paid__min']

In [10]:
#with open(os.path.join(feature_selection_model_output_path, 'keep_features_national_model.json'), "w") as json_file:
#    json.dump(keep_features, json_file, indent=4)

In [45]:
#mkdir /d/shared/users/jag

In [47]:
#mkdir /d/shared/users/jag/penfed

In [48]:
#mkdir /d/shared/users/jag/penfed/autoindirectv1

In [71]:
#mkdir /d/shared/users/jag/penfed/autoindirectv1/test_only_5_features

In [9]:
#ls /d/shared/users/jag

penfed/


## Let's Create a test with only 5 features

In [28]:
keep_features_to_keep = {'inq': ['inq_inquiries__sum']}

with open(os.path.join('/d/shared/users/jag/penfed/autoindirectv1/test_only_5_features', 'keep_features_national_model.json'), "w") as json_file:
    json.dump(keep_features_to_keep, json_file, indent=4)

In [29]:
national_model_id = 'nationalmodel3test'
national_model_output_path = f'/d/shared/users/jag/penfed/autoindirectv1/{national_model_id}'
national_model_output_path

'/d/shared/users/jag/penfed/autoindirectv1/nationalmodel3test'

In [21]:
rm -rf /d/shared/users/jag/penfed/autoindirectv1/nationalmodel3test*

^C


# We changed to only grab the target and collect table, 
## Onyl the collection features: f"/d/shared/users/jag/penfed/autoindirectv1/test_only_5_features/keep_features_national_model.json"
and data split from 
```python
data_split: {
          "train": {
            "start_date": "2016-04-01",
            "end_date": "2021-01-01"
          },
          "valid": {
            "start_date": "2021-01-01",
            "end_date": "2021-04-01"
          },
          "test": {
            "start_date": "2021-04-01",
            "end_date": "2022-01-01"
          }
```


to 

```python
data_split: {
          "train": {
            "start_date": "2016-04-01",
            "end_date": "2016-07-01"
          },
          "valid": {
            "start_date": "2016-07-01",
            "end_date": "2016-08-01"
          },
          "test": {
            "start_date": "2016-08-01",
            "end_date": "2016-10-01"
          }
```

# * NOTE TO DO FILTERERING WE NEED THE APP TABLE SO WE NEED TO ADD THAT TO TABLES ELSE WE WOULD HAVE GOTTEN PROBLEMS BECAUSE S3PowerSampler [EQUIRES IT](https://docs.google.com/document/d/1dTmERQQvpJlXnNtXXOWgL6Kyo1Ks9poNn3qSwIsSYSM/edit?tab=t.0) 

In [30]:
config = {
  "model_type": "standard_model",
  "storage_location": "s3",
  "lookalike": {
    "bureau": "normalized",
    "accounts":[
      "auto"
    ],
    "splits": [
      "train",
      "valid",
      "test"
    ],
    "pull_names": None,
    "states": None,
    "max_row": None,
    "tables": ['inq', 'target', 'app'],
  },
  "model_config": {
      "data_split": {
          "train": {
            "start_date": "2016-04-01",
            "end_date": "2016-05-01"
          },
          "valid": {
            "start_date": "2016-05-01",
            "end_date": "2016-06-01"
          },
          "test": {
            "start_date": "2016-06-01",
            "end_date": "2016-08-01"
          }
    },
    "target": "final_DQ60_m24",
    "base_features": f"/d/shared/users/jag/penfed/autoindirectv1/test_only_5_features/keep_features_national_model.json",
#    "feature_filter_rules": "/home/lyt/CODEBASE/model-engine_penfed/model-engine/model_engine/assets/power/feature_rules/intuitive_features.json",
    "memory_efficient": True,
    "pipeline_factory": None,
    "bivariate_fe_instructions": [],
    "monotonic_constraints_list": [],
    "exclusion_list": [],
    "feature_definition_list": [],
    "key_factor_mapping_list": [],
    "fold_valid": True,
    "mandatory_features": []
  }
}

### also changed it to dhared

## Initialization

We first initialize 
```python
manifest_template = load_asset('power/artifact_manifest_template.json')
manifest = DictHelper(manifest_template).replace_value_segment('path', 'data=SOURCE', 'data=national')
auto_valid_pipeline = load_asset('pipeline_factory/auto_valid_pipeline_factory.json')

def __init__(self, configuration, model_output_path, overwrite_model_output, spark_s3_path=None, profile_name=None):
    '''
    model_output_path: Final model output path in dshared or s3. Defines the final place where our zaml object models are stored.
    spark_s3_path: used to store the temporary results of spark training. Temporary location that may be deleted or overwritten after running the code.
    '''
    super().__init__(configuration, model_output_path)
    self.overwrite_model_output = overwrite_model_output
    self.spark_s3_path = spark_s3_path
    self.profile_name = profile_name
```

# Manifest_template is found [here](https://github.com/Katlean/model-engine/blob/c78ae3f22054dbc9b08b4c0540318ee0976ec889/model_engine/assets/power/artifact_manifest_template.json).
* I believe this sets up locations where we are going to save the output files
* When we call the second code that just replace data=Source with data equals national
* * We can see that [auto_valid_pipeline](https://github.com/Katlean/model-engine/blob/c78ae3f22054dbc9b08b4c0540318ee0976ec889/model_engine/assets/pipeline_factory/auto_pipeline_factory.json) is here and it tells us the steps we will go through. Basically we will
  a) level_selection: For each categorical variable, all levels with less than .01 of the values mapped to other. Then we one-hot encode so we create binary variable for each level
  b) We fill na with -1
  c) We do correlation feature selection dropping any features that have more than .995 correlation with any other
  d) We do gain feature selection based on these [params](https://github.com/Katlean/model-engine/blob/c78ae3f22054dbc9b08b4c0540318ee0976ec889/model_engine/assets/pipeline_factory/auto_valid_pipeline_factory.json). We split our training set into train and valid (85/15 randomly) **ASK ABOUT THIS PART/COnfirm later). we then train a baseline model to get the top 900, and since n_iter is not provided, the default is 10, this means we run 10 iterations keeping on dropping 20 features each until we get to 100 features. We then choose the feature set with the [highest score](https://github.com/Katlean/zaml/blob/e4c61a6d16b70ed4e21b7704a936eb624d9dd318/zaml/model/feature_selection/gain_feature_selection.py#L351C8-L352C44).
# I don't see anything in the output about pipeline factory looks like it does not do the feature selction

## We intiialize the rest in [power_modeling_base](https://github.com/Katlean/model-engine/blob/c78ae3f22054dbc9b08b4c0540318ee0976ec889/model_engine/power/post_sale/base/power_modeling_base.py#L22C5-L54C35)

```python
possible_bureaus = {
                    "equifax": load_asset('power/equifax.json'),
                    "transunion": load_asset('power/transunion.json'),
                    "experian": load_asset('power/experian.json'),
                    "normalized": load_asset('power/normalized.json')
                    }

    power_asset = load_asset('power/power_asset.json')
    
    @classmethod
    def get_clients(cls):
        return ClientHandler.get_clients()
    
    def __init__(self, config, artifacts_path):
        self.config = config
        self.schema = config.get('schema', 'v3')
        self.storage_location = config.get('storage_location', 'dshared')
        self.output_system = 's3' if artifacts_path.startswith('s3://') else 'dshared'
        self.model_config = config['model_config']
        self.data_split = self.model_config.get('data_split')
        self.lookalike = config.get('lookalike', {})
        self.bureau = self.get_bureau_name(config)
        self.bureau_asset = self.get_bureau_asset()
        if self.storage_location == 'dshared':
            self.lookalike_obj = LookalikeHandler(self.data_split, **self.lookalike)
        else:
            self.lookalike_obj = S3LookalikeHandler(self.data_split, **self.lookalike)
        self.artifacts_path = artifacts_path
        self.model_constraint_obj = ModelConstraints(self.config)

        self.enforce_artifact_validation = self.model_config.get('enforce_artifact_validation', True) 

        self.client_handler = None
```

* Note storage location is where we load the data
* look at rest later
## Key HEre is that our data_split is taken from config['model_config']['data_split'] and that along with the lookalike gives us our [S3LookalikeHandler to grab the data](https://github.com/Katlean/model-engine/blob/38811522d8839d8bc8b547d68c03f336a78f6f8a/model_engine/power/post_sale/base/power_modeling_base.py#L52C13-L52C87)

# We can see below the features we started with for this version (the buraeu features from the pen-fed origence model and (keep_national_features) and then the features we ended with for the national model

In [ ]:
ls /d/shared/silver_projects_v2/penfed/autoindirectv1/modeling/clientmodel3a/

In [43]:
keep_features = json.load(open('/d/shared/silver_projects_v2/penfed/autoindirectv1/modeling/clientmodel3a/keep_features.json'))
keep_features.pop('client_data')

['client_data_LTV_NORM']

In [47]:
og_nat_feat_list = []
for v in keep_features.values():
    og_nat_feat_list.extend(v)
len(og_nat_feat_list)

269

In [49]:
og_feat_import = pd.read_parquet('/d/shared/silver_projects_v2/penfed/autoindirectv1/modeling/nationalmodels/nationalmodel3/feature_importance.parquet')

In [56]:
best_model_params  = json.load(open('/d/shared/silver_projects_v2/penfed/autoindirectv1/modeling/nationalmodels/nationalmodel3/asset.json'))

In [59]:
best_model_params['config'].keys()

dict_keys(['data_split', 'target', 'base_features', 'memory_efficient', 'pipeline_factory', 'bivariate_fe_instructions', 'monotonic_constraints_list', 'exclusion_list', 'feature_definition_list', 'key_factor_mapping_list', 'fold_valid', 'mandatory_features'])

In [ ]:
best_model_params['config']['pipeline_factory']

In [60]:
best_model_params['config']['pipeline_factory']

{'transformers': [{'zaml_class': 'FillNA',
   'params': {'replace_by': -1, 'add_flags': False}},
  {'zaml_class': 'GeneralFeatureSelection',
   'params': {'features': ['app_is_client']}}],
 'model': {'zaml_class': 'XGBoostModel',
  'params': {'n_estimators': 15882,
   'learning_rate': 0.02,
   'max_depth': 6,
   'subsample': 0.9,
   'scale_pos_weight': 4,
   'colsample_bytree': 0.25,
   'min_child_weight': 150,
   'seed': 12,
   'reg_lambda': 5,
   'eval_metric': 'auc',
   'verbose_eval': 100,
   'enable_index_matching': True,
   'tree_method': 'hist',
   'n_min_estimators': 0,
   'booster': 'gbtree',
   'gamma': 0,
   'max_delta_step': 0,
   'colsample_bylevel': 1,
   'reg_alpha': 0,
   'max_leaves': 0,
   'base_score': 0.5}}}

In [48]:
ls /d/shared/silver_projects_v2/penfed/autoindirectv1/modeling/nationalmodels/nationalmodel3

app/                          overall_scores/
artifact_manifest.json        overall_zest_scores/
asset.json                    parsed_monotonic_constraints_list.json
best_model_params.json        pipeline.obj
calibration_object.obj        score_recalibration_mapping.json
client_test_scores_1.parquet  splitter.obj
client_test_scores_2.parquet  static_asset.json
data_description.json         supporting_model/
feature_definition.parquet    target/
feature_importance.parquet    test_auc.json
fe_data/                      test_data_summary.json
fit_time_info.json            test_ks.json
injected_model_config.json    top_features.parquet
keep_features.json            train_auc.json
key_factors_mapping.json      train_data_summary.json
lookalike_sample/             train_history.json
model.obj                     train_ks.json
model_strategy.json           unfold_data_description.json
mrm_pipeline.obj              value_based_key_factor_mapping.json
national_model_config.json    versions.json

## Running the init

In [31]:
nmb = NationalModelBuilder(
    configuration=config,
    model_output_path=national_model_output_path,
    overwrite_model_output=True
)

In [ ]:
nmb.auto_valid_pipeline

In [64]:
nmb.__dict__

{'config': {'model_type': 'standard_model',
  'storage_location': 's3',
  'lookalike': {'bureau': 'normalized',
   'accounts': ['auto'],
   'splits': ['train', 'valid', 'test'],
   'pull_names': None,
   'states': None,
   'max_row': None,
   'tables': ['inq', 'target', 'app']},
  'model_config': {'data_split': {'train': {'start_date': '2016-04-01',
     'end_date': '2016-05-01'},
    'valid': {'start_date': '2016-05-01', 'end_date': '2016-06-01'},
    'test': {'start_date': '2016-06-01', 'end_date': '2016-08-01'}},
   'target': 'final_DQ60_m24',
   'base_features': '/d/shared/users/jag/penfed/autoindirectv1/test_only_5_features/keep_features_national_model.json',
   'memory_efficient': True,
   'pipeline_factory': None,
   'bivariate_fe_instructions': [],
   'monotonic_constraints_list': [],
   'exclusion_list': [],
   'feature_definition_list': [],
   'key_factor_mapping_list': [],
   'fold_valid': True,
   'mandatory_features': []}},
 'schema': 'v3',
 'storage_location': 's3',
 'out

In [20]:
nmb.manifest

{'data': {'train_app': {'type': 'dataframe',
   'path': 'app/split=train/data=national/data_0.parquet',
   'required': False},
  'valid_app': {'type': 'dataframe',
   'path': 'app/split=valid/data=national/data_0.parquet',
   'required': False},
  'test_app': {'type': 'dataframe',
   'path': 'app/split=test/data=national/data_0.parquet',
   'required': False},
  'train_target': {'type': 'series',
   'path': 'target/split=train/data=national/data_0.parquet',
   'required': False},
  'valid_target': {'type': 'series',
   'path': 'target/split=valid/data=national/data_0.parquet',
   'required': False},
  'test_target': {'type': 'series',
   'path': 'target/split=test/data=national/data_0.parquet',
   'required': False},
  'train_fe_data': {'type': 'dataframe',
   'path': 'fe_data/split=train/data=national/data_0.parquet',
   'required': False},
  'valid_fe_data': {'type': 'dataframe',
   'path': 'fe_data/split=valid/data=national/data_0.parquet',
   'required': False},
  'test_fe_data': {

# After Intiailzed we call [run](https://github.com/Katlean/model-engine/blob/c78ae3f22054dbc9b08b4c0540318ee0976ec889/model_engine/power/post_sale/national_model_builder.py#L108C5-L110C90) which 
```python
 def run(self):
        self.validate_arguments()
        super().archive_config(self.config, 'national_model_config.json', self.artifacts_path)
        self.inject_model_config()
        super().archive_config(self.config, 'injected_model_config.json', self.artifacts_path)  # to track what's happening at archive_input_config
        self.build_national_model()
```

In [32]:
nmb.validate_arguments()

In [33]:
super(NationalModelBuilder, nmb).archive_config(
    nmb.config, 
    'national_model_config.json', 
    nmb.artifacts_path
)

In [68]:
ls /d/shared/users/jag/penfed/autoindirectv1/nationalmodel3test

national_model_config.json


In [69]:
national_model_config = json.load(open(f'test/nationalmodel3/national_model_config.json'))

In [70]:
nmb.config

{'model_type': 'standard_model',
 'storage_location': 's3',
 'lookalike': {'bureau': 'normalized',
  'accounts': ['auto'],
  'splits': ['train', 'valid', 'test'],
  'pull_names': None,
  'states': None,
  'max_row': None,
  'tables': ['inq', 'target', 'app']},
 'model_config': {'data_split': {'train': {'start_date': '2016-04-01',
    'end_date': '2016-05-01'},
   'valid': {'start_date': '2016-05-01', 'end_date': '2016-06-01'},
   'test': {'start_date': '2016-06-01', 'end_date': '2016-08-01'}},
  'target': 'final_DQ60_m24',
  'base_features': '/d/shared/users/jag/penfed/autoindirectv1/test_only_5_features/keep_features_national_model.json',
  'memory_efficient': True,
  'pipeline_factory': None,
  'bivariate_fe_instructions': [],
  'monotonic_constraints_list': [],
  'exclusion_list': [],
  'feature_definition_list': [],
  'key_factor_mapping_list': [],
  'fold_valid': True,
  'mandatory_features': []}}

In [71]:
national_model_config

{'model_type': 'standard_model',
 'storage_location': 's3',
 'lookalike': {'bureau': 'normalized',
  'accounts': ['auto'],
  'splits': ['train', 'valid', 'test'],
  'pull_names': None,
  'states': None,
  'max_row': None,
  'tables': ['trade', 'inq', 'bnkr', 'collec', 'target', 'app', 'member']},
 'model_config': {'data_split': {'train': {'start_date': '2016-04-01',
    'end_date': '2021-01-01'},
   'valid': {'start_date': '2021-01-01', 'end_date': '2021-04-01'},
   'test': {'start_date': '2021-04-01', 'end_date': '2022-01-01'}},
  'target': 'final_DQ60_m24',
  'base_features': '/d/shared/silver_projects_v2/penfed/autoindirectv1/modeling/clientmodel3a/keep_features_national_model.json',
  'memory_efficient': True,
  'pipeline_factory': None,
  'bivariate_fe_instructions': [],
  'monotonic_constraints_list': [],
  'exclusion_list': [],
  'feature_definition_list': [],
  'key_factor_mapping_list': [],
  'fold_valid': True,
  'mandatory_features': []}}

In [72]:
config['lookalike']

{'bureau': 'normalized',
 'accounts': ['auto'],
 'splits': ['train', 'valid', 'test'],
 'pull_names': None,
 'states': None,
 'max_row': None,
 'tables': ['inq', 'target', 'app']}

In [28]:
config['model_config']

{'data_split': {'train': {'start_date': '2016-04-01',
   'end_date': '2016-05-01'},
  'valid': {'start_date': '2016-05-01', 'end_date': '2016-06-01'},
  'test': {'start_date': '2016-06-01', 'end_date': '2016-08-01'}},
 'target': 'final_DQ60_m24',
 'base_features': '/d/shared/users/jag/penfed/autoindirectv1/test_only_5_features/keep_features_national_model.json',
 'memory_efficient': True,
 'pipeline_factory': None,
 'bivariate_fe_instructions': [],
 'monotonic_constraints_list': [],
 'exclusion_list': [],
 'feature_definition_list': [],
 'key_factor_mapping_list': [],
 'fold_valid': True,
 'mandatory_features': []}

## Before inject_model_config we don't have a pipeline factory value

Since our model type is not feature selection, our pipeline factory will be buraeu pipeline factory
# This is found right [here](https://github.com/Katlean/model-engine/blob/c78ae3f22054dbc9b08b4c0540318ee0976ec889/model_engine/assets/pipeline_factory/bureau_pipeline_factory.json)
# **QUESTION: WHAT ABOUT FOLD_VALID AND NESTIMATORS**
* But I thought for fold-in we estimate the estimators
```python
{
    "name": "BureauPipelineFactory",
    "transformers": [
        {
            "zaml_class": "FillNA",
            "params": {
                "replace_by": -1,
                "add_flags": false
            }
        },
        {
            "zaml_class": "GeneralFeatureSelection",
            "params": {
                "features": [
                    "app_is_client"
                ]
            }
        }
    ],
    "model": {
        "zaml_class": "XGBoostModel",
        "params": {
            "n_estimators": 25000,
            "learning_rate": 0.02,
            "max_depth": 6,
            "subsample": 0.9,
            "scale_pos_weight": 4,
            "colsample_bytree": 0.25,
            "min_child_weight": 150,
            "seed": 12,
            "early_stopping_rounds": 200,
            "reg_lambda": 5,
            "eval_metric": "auc",
            "verbose_eval": 100,
            "enable_index_matching": true,
            "tree_method": "hist",
             "n_min_estimators": 0,
             "booster": "gbtree",
             "gamma": 0,
             "max_delta_step": 0,
             "colsample_bylevel": 1,
             "reg_alpha": 0,
             "max_leaves": 0,
             "base_score": 0.5
        }
    }
}

In [75]:
nmb.config['model_config']['pipeline_factory']

In [34]:
nmb.inject_model_config()

In [77]:
nmb.config['model_config']['pipeline_factory']

'BureauPipelineFactory'

In [35]:
super(NationalModelBuilder, nmb).archive_config(
    nmb.config, 
    'injected_model_config.json', 
    nmb.artifacts_path
)

In [33]:
injected_model_config  = json.load(open(f'/d/shared/users/jag/penfed/autoindirectv1/nationalmodel3test/injected_model_config.json'))

In [34]:
injected_model_config

{'model_type': 'standard_model',
 'storage_location': 's3',
 'lookalike': {'bureau': 'normalized',
  'accounts': ['auto'],
  'splits': ['train', 'valid', 'test'],
  'pull_names': None,
  'states': None,
  'max_row': None,
  'tables': ['inq', 'target', 'app']},
 'model_config': {'data_split': {'train': {'start_date': '2016-04-01',
    'end_date': '2016-05-01'},
   'valid': {'start_date': '2016-05-01', 'end_date': '2016-06-01'},
   'test': {'start_date': '2016-06-01', 'end_date': '2016-08-01'}},
  'target': 'final_DQ60_m24',
  'base_features': '/d/shared/users/jag/penfed/autoindirectv1/test_only_5_features/keep_features_national_model.json',
  'memory_efficient': True,
  'pipeline_factory': 'BureauPipelineFactory',
  'bivariate_fe_instructions': [],
  'monotonic_constraints_list': [],
  'exclusion_list': [],
  'feature_definition_list': [],
  'key_factor_mapping_list': [],
  'fold_valid': True,
  'mandatory_features': []}}

In [35]:
ls /d/shared/users/jag/penfed/autoindirectv1/nationalmodel3test

injected_model_config.json  national_model_config.json


## We then call build [build_national_model](https://github.com/Katlean/model-engine/blob/c78ae3f22054dbc9b08b4c0540318ee0976ec889/model_engine/power/post_sale/national_model_builder.py#L298) which calls 

```python
if self.config['model_config']['fold_valid']:
    # Initial model on full data split
    supporting_model_path = os.path.join(self.artifacts_path, 'supporting_model')
    print(f'Attempting to create a national model with full data split here {supporting_model_path}')
    self._build_full_split_model(self.config['model_config'], supporting_model_path, self.manifest)
    # 2nd model by folding the validation into training set
    print(f'Attempting to create a national model by folding validation into training here {self.artifacts_path}')
    self._build_reduced_split_model(self.artifacts_path, supporting_model_path, self.manifest)
else:
    print(f'Attempting to create a national model here {self.artifacts_path}') 
    self._build_full_split_model(self.config['model_config'], self.artifacts_path, self.manifest)
if 'megamodel_config_options' in self.config['model_config']:
    self._archive_select_pipeline(self.artifacts_path)
if self.config['model_type'] == 'feature_selection':
    self._archive_MRM_artifacts(self.artifacts_path)
```

In [36]:
supporting_model_path = os.path.join(nmb.artifacts_path, 'supporting_model')

In [37]:
supporting_model_path

'/d/shared/users/jag/penfed/autoindirectv1/nationalmodel3test/supporting_model'

### This calls [build_full_split_model](https://github.com/Katlean/model-engine/blob/c78ae3f22054dbc9b08b4c0540318ee0976ec889/model_engine/power/post_sale/national_model_builder.py#L510)

```python
def _build_full_split_model(self, model_config, model_path, manifest):
        data_section = super().sample_lookalike(os.path.join(model_path, 'lookalike_sample'))
        data_section = super().normalize_io(data_section=data_section, memory_efficient=self.config['model_config'].get('memory_efficient'),ignore_sources=['config'])
        if 'megamodel_config_options' in self.config['model_config']:
            data_section = self._inject_megamodel_configuraton_asset(data_section, self.config['model_config']['megamodel_config_options'])
        mb_asset = super().make_mb_asset(data_section, model_config)
        build_model(mb_asset, model_path, manifest=manifest)
        self._add_data_description(model_path, model_path, 'data_description.json')
```
Note that sample_looklalike is


# look into this does it only sample from the tables you actually use?

```python
 ps_config = self.lookalike_obj.power_sample_config
ps_output_path = os.path.join(self.artifacts_path, 'lookalike_sample')
if self.storage_location == 'dshared':
    return power_sampler(ps_config, archive_path=ps_output_path)
else:
    return S3PowerSampler(ps_config, archive_path=ps_output_path).generate_sample()
```

Note when we create lookalike manager we [take in the data_split and the lookalike keys](https://github.com/Katlean/model-engine/blob/c78ae3f22054dbc9b08b4c0540318ee0976ec889/model_engine/power/post_sale/base/power_modeling_base.py#L48C13-L48C87)

Most of the [initilization](https://github.com/Katlean/model-engine/blob/c78ae3f22054dbc9b08b4c0540318ee0976ec889/model_engine/power/post_sale/base/s3_lookalike_handler.py#L7C1-L13C37) happens with lookalikeHandler it's parent class
```python
class S3LookalikeHandler(LookalikeHandler):
    def __init__(self, data_split, bureau: Literal['equifax', 'experian', 'transunion', 'normalized']='equifax', 
                 tables=None, states=None, accounts=None, pull_names=None, splits=None, kob=None, max_row=None, 
                 filter_table=None, use_filter_columns=None, partition=None):
        super().__init__(data_split, bureau, tables, states, accounts, pull_names, splits, kob, max_row, 
                         filter_table, use_filter_columns, partition)
        self.pull_names = pull_names
```
This occurs in [power/post_sale_base/lookalike_handler.py](https://github.com/Katlean/model-engine/blob/c78ae3f22054dbc9b08b4c0540318ee0976ec889/model_engine/power/post_sale/base/lookalike_handler.py)

```python
def __init__(self,  data_split, bureau='transunion', tables=None, states=None, accounts=None,
                 pull_names=None, splits=None, kob=None, max_row=None, filter_table=None, use_filter_columns=None,
                 partition=None):
        
        self.bureau = bureau
        self.partition = partition
        self.base_data_path = os.path.join(self.power_asset["data_path"]["file_system"], f"BUREAU={self.bureau}")
        self.tables = tables or ['trade', 'inq', 'bnkr', 'collec', 'target', 'app']
        self.bureau_asset = self._get_bureau_asset(bureau)
        self.loan_type_mapping = self.bureau_asset['loan_type']
        self.partition_mapping = self.bureau_asset["partition"]

        self.sampler_data_section = self._make_sampler_data_section(self.bureau_asset, self.tables)
        self.data_split = data_split
        self.states = states # verify
        self.accounts = accounts # verify
        self.pull_names = pull_names or self.bureau_asset["pull_name"]["dshared"]["train"]
        self.splits = splits or ['train', 'valid']
        self.kob = kob
        if (self.kob is not None) and (not isinstance(self.kob, list)):
            self.kob = [self.kob]
        self.max_row = max_row
        self.filter_table = filter_table

```





In [90]:
nmb.lookalike_obj.__dict__.keys()

dict_keys(['bureau', 'partition', 'base_data_path', 'tables', 'bureau_asset', 'loan_type_mapping', 'partition_mapping', 'sampler_data_section', 'data_split', 'states', 'accounts', 'pull_names', 'splits', 'kob', 'max_row', 'filter_table'])

In [91]:
nmb.lookalike_obj.__dict__['base_data_path']

'/d/shared/silver_projects_v2/project_power/shared_data/processed/national_data/BUREAU=normalized'

In [92]:
nmb.lookalike_obj.__dict__['tables']

['inq', 'target', 'app']

In [93]:
nmb.lookalike_obj.__dict__['loan_type_mapping']

{'auto': ['00', '3A', 'AU', 'AL'],
 'credit_card': ['07', '0G', '18', '2A', 'CC', 'CH', 'FX', 'SC'],
 'personal': ['01',
  '02',
  '03',
  '06',
  '0A',
  '0F',
  '11',
  '13',
  '15',
  '20',
  '29',
  '47',
  '4D',
  '70',
  '7B',
  '92',
  '91',
  'US',
  'LS',
  'LC',
  'SE',
  'PS',
  'IS',
  'TS',
  'LN',
  'RD',
  'LE',
  'NT',
  'RA',
  'CU',
  'GO',
  'AX',
  'UC',
  'CO'],
 'homeequity': ['89',
  '6D',
  '22',
  '23',
  '04',
  '05',
  '5B',
  '9A',
  'HE',
  'SH',
  'SO',
  'FI',
  'SM',
  'HI',
  'SI'],
 'businessloan': ['BU', 'BL', 'BC', '8A', '9B', '1B', '10']}

In [83]:
config['lookalike']

{'bureau': 'normalized',
 'accounts': ['auto'],
 'splits': ['train', 'valid', 'test'],
 'pull_names': None,
 'states': None,
 'max_row': None,
 'tables': ['inq', 'target', 'app']}

In [42]:
nmb.lookalike_obj.power_sample_config

{'data': {'partition': None,
  'asset': {'inq': 'equifax/cms_6/inquiry.json',
   'app': {'info': {'key': 'ZEST_KEY', 'app_date': 'appDate'},
    'table_name': 'app',
    'table_type': 'one_to_one',
    'feature_engineering': 'one_to_one'},
   'target': {'info': {'col': 'final_DQ60_m24', 'key': 'ZEST_KEY'},
    'table_type': 'target'}}},
 'sample': [{'BUREAU': 'experian',
   'DATE_OF_REQUEST': {'start_date': '2016-04-01', 'end_date': '2016-05-01'},
   'ACCOUNT_TYPE_CODE': ['00', '3A'],
   'PULL_NAME': ['national_1.3', 'national_2'],
   'GEO_STATE_CODE': None},
  {'BUREAU': 'equifax',
   'DATE_OF_REQUEST': {'start_date': '2016-04-01', 'end_date': '2016-05-01'},
   'ACCOUNT_TYPE_CODE': ['00', '3A'],
   'PULL_NAME': ['national_1.1',
    'national_2',
    'national_1_home_equity',
    'national_3'],
   'GEO_STATE_CODE': None},
  {'BUREAU': 'transunion',
   'DATE_OF_REQUEST': {'start_date': '2016-04-01', 'end_date': '2016-05-01'},
   'ACCOUNT_TYPE_CODE': ['AU', 'AL'],
   'PULL_NAME': ['natio

In [80]:
nmb.lookalike_obj

# and that normalize_io is in [power_modeling_base](https://github.com/Katlean/model-engine/blob/38811522d8839d8bc8b547d68c03f336a78f6f8a/model_engine/power/post_sale/base/power_modeling_base.py#L109C6-L114C28)

# note we call the schema part to get the keep_features

```python
def normalize_io(self, data_section, memory_efficient=False, ignore_sources=None):
    '''
    This method inject the keep_features under the io_params in the data asset.
    '''
    data_section = normalize_io(data_section, self.config['model_config'], keep_features=list(schema[self.schema].keys()), memory_efficient=memory_efficient, ignore_sources=ignore_sources)
    return data_section
```

which wraps around in [model_engine/power/post_sale/base/utils.py](https://github.com/Katlean/model-engine/blob/master/model_engine/power/post_sale/base/utils.py) 

```python
def normalize_io(data_section, model_config, keep_features, memory_efficient:bool=False, ignore_sources=None):
    ignore_sources = ignore_sources or []
    data_section['app']['io_params']['keep_features'] = keep_features
    data_section = normalize_target(data_section, NORMALIZE_TARGET_TABLES)
    if 'target_spec' in model_config:
        data_section['target']['io_params']['keep_features'] = ['ZEST_KEY', model_config['target_spec']['target_col']]
        data_section['target']['asset']['info']['col'] = model_config['target_spec']['target_col']
    else:
        # backward compatibility
        data_section['target']['io_params']['keep_features'] = ['ZEST_KEY', model_config['target']]
        data_section['target']['asset']['info']['col'] = model_config['target']
    if 'base_features' in model_config:
        with open(model_config['base_features'], 'r') as infile:
            base_features = json.load(infile)
        for data, features in base_features.items():
            if data not in ignore_sources:
                data_section[data]['io_params']['keep_features'] = features
    # For all tables included in data_section, introduce the option of "memory_efficient"
    for table in data_section.keys():
        if 'io_params' in data_section[table]:
            data_section[table]['io_params']['memory_efficient'] = memory_efficient
    return data_section
```

## Looks like the data_section create the data by sampling

## What is it actually doing here?

In [38]:
data_section = super(NationalModelBuilder, nmb).sample_lookalike(
    os.path.join(supporting_model_path, 'lookalike_sample')
)

Sampling experian, national_1.3
Sampling experian, national_2
Sampling equifax, national_1.1
Sampling equifax, national_2
Sampling equifax, national_1_home_equity
Sampling equifax, national_3
Sampling transunion, national_3_refresh
Sampling transunion, national_4
Sampling experian, national_1.3
Sampling experian, national_2
Sampling equifax, national_1.1
Sampling equifax, national_2
Sampling equifax, national_1_home_equity
Sampling equifax, national_3
Sampling transunion, national_3_refresh
Sampling transunion, national_4
Sampling experian, national_1.3
Sampling experian, national_2
Sampling equifax, national_1.1
Sampling equifax, national_2
Sampling equifax, national_1_home_equity
Sampling equifax, national_3
Sampling transunion, national_3_refresh
Sampling transunion, national_4



/home/jag/.conda/envs/demoanalysis/lib/python3.10/site-packages/fsspec/registry.py:301: UserWarning: Your installed version of s3fs is very old and known to cause
severe performance issues, see also https://github.com/dask/dask/issues/10276

To fix, you should specify a lower version bound on s3fs, or
update the current installation.

  warnings.warn(s3_msg)
/home/jag/.conda/envs/demoanalysis/lib/python3.10/site-packages/fsspec/registry.py:301: UserWarning: Your installed version of s3fs is very old and known to cause
severe performance issues, see also https://github.com/dask/dask/issues/10276

To fix, you should specify a lower version bound on s3fs, or
update the current installation.

  warnings.warn(s3_msg)
/home/jag/.conda/envs/demoanalysis/lib/python3.10/site-packages/fsspec/registry.py:301: UserWarning: Your installed version of s3fs is very old and known to cause
severe performance issues, see also https://github.com/dask/dask/issues/10276

To fix, you should specify a lower 

## When we sample_lookalike we create the data section which points to where the files are saved

# We can see calling sample_look_alike made the data section which tells us which files we sampled from the national data
## It basically creates json files for each of the tables we sampled from telling which files we need to grab



In [45]:
data_section

{'inq': {'asset': 'equifax/cms_6/inquiry.json',
  'data': '#/d/shared/users/jag/penfed/autoindirectv1/nationalmodel3test/lookalike_sample/inq_sampled_files.json',
  'io_params': {'drop_duplicates': True,
   'keep_keys': '/d/shared/users/jag/penfed/autoindirectv1/nationalmodel3test/lookalike_sample/keep_keys.json'}},
 'app': {'asset': {'info': {'key': 'ZEST_KEY', 'app_date': 'appDate'},
   'table_name': 'app',
   'table_type': 'one_to_one',
   'feature_engineering': 'one_to_one'},
  'data': '#/d/shared/users/jag/penfed/autoindirectv1/nationalmodel3test/lookalike_sample/app_sampled_files.json',
  'io_params': {'drop_duplicates': True,
   'keep_keys': '/d/shared/users/jag/penfed/autoindirectv1/nationalmodel3test/lookalike_sample/keep_keys.json'}},
 'target': {'asset': {'info': {'col': 'final_DQ60_m24', 'key': 'ZEST_KEY'},
   'table_type': 'target'},
  'data': '#/d/shared/users/jag/penfed/autoindirectv1/nationalmodel3test/lookalike_sample/target_sampled_files.json',
  'io_params': {'drop_d

In [94]:
data_inq = json.load(open('/d/shared/users/jag/penfed/autoindirectv1/nationalmodel3test/lookalike_sample/inq_sampled_files.json'))

In [96]:
data_inq[0:4]

['s3://power-experian-prod/NATIONAL/PROCESSED/DATA/BUREAU=experian/FORMAT=arf7/TABLE=inquiry_fe/PULL_NAME=national_1.3/FEW_VERSION=0.0.9/ME_VERSION=v1.8.0/DATE_OF_REQUEST=2016-03-31/ACCOUNT_TYPE_CODE=00/GEO_STATE_CODE=AK/189cfec7493844348c14b6c2c2acf893.parquet',
 's3://power-experian-prod/NATIONAL/PROCESSED/DATA/BUREAU=experian/FORMAT=arf7/TABLE=inquiry_fe/PULL_NAME=national_1.3/FEW_VERSION=0.0.9/ME_VERSION=v1.8.0/DATE_OF_REQUEST=2016-03-31/ACCOUNT_TYPE_CODE=00/GEO_STATE_CODE=AL/f4e62e12bf964c26a7bc25e91f8a5ff3.parquet',
 's3://power-experian-prod/NATIONAL/PROCESSED/DATA/BUREAU=experian/FORMAT=arf7/TABLE=inquiry_fe/PULL_NAME=national_1.3/FEW_VERSION=0.0.9/ME_VERSION=v1.8.0/DATE_OF_REQUEST=2016-03-31/ACCOUNT_TYPE_CODE=00/GEO_STATE_CODE=AR/ce52697ebadf41bf951cf4baeb6af145.parquet',
 's3://power-experian-prod/NATIONAL/PROCESSED/DATA/BUREAU=experian/FORMAT=arf7/TABLE=inquiry_fe/PULL_NAME=national_1.3/FEW_VERSION=0.0.9/ME_VERSION=v1.8.0/DATE_OF_REQUEST=2016-03-31/ACCOUNT_TYPE_CODE=00/GEO_S

## We can see data section points to location we will use to train on for the actual tables we pointed to in lookalike when we created the config

## Look into what normalie io does later

In [105]:
data_section['inq']['io_params']

{'drop_duplicates': True,
 'keep_keys': '/d/shared/users/jag/penfed/autoindirectv1/nationalmodel3test/lookalike_sample/keep_keys.json'}

In [106]:
data_section['target']['io_params']

{'drop_duplicates': True,
 'keep_keys': '/d/shared/users/jag/penfed/autoindirectv1/nationalmodel3test/lookalike_sample/keep_keys.json'}

In [104]:
data_section['app']['io_params']

{'drop_duplicates': True,
 'keep_keys': '/d/shared/users/jag/penfed/autoindirectv1/nationalmodel3test/lookalike_sample/keep_keys.json'}

In [110]:
data_section['target']

{'asset': {'info': {'col': 'final_DQ60_m24', 'key': 'ZEST_KEY'},
  'table_type': 'target'},
 'data': '#/d/shared/users/jag/penfed/autoindirectv1/nationalmodel3test/lookalike_sample/target_sampled_files.json',
 'io_params': {'drop_duplicates': True,
  'keep_keys': '/d/shared/users/jag/penfed/autoindirectv1/nationalmodel3test/lookalike_sample/keep_keys.json'}}

In [109]:
data_section['inq']

{'asset': 'equifax/cms_6/inquiry.json',
 'data': '#/d/shared/users/jag/penfed/autoindirectv1/nationalmodel3test/lookalike_sample/inq_sampled_files.json',
 'io_params': {'drop_duplicates': True,
  'keep_keys': '/d/shared/users/jag/penfed/autoindirectv1/nationalmodel3test/lookalike_sample/keep_keys.json'}}

In [108]:
data_section['app'] 

{'asset': {'info': {'key': 'ZEST_KEY', 'app_date': 'appDate'},
  'table_name': 'app',
  'table_type': 'one_to_one',
  'feature_engineering': 'one_to_one'},
 'data': '#/d/shared/users/jag/penfed/autoindirectv1/nationalmodel3test/lookalike_sample/app_sampled_files.json',
 'io_params': {'drop_duplicates': True,
  'keep_keys': '/d/shared/users/jag/penfed/autoindirectv1/nationalmodel3test/lookalike_sample/keep_keys.json'}}

In [39]:
data_section = super(NationalModelBuilder, nmb).normalize_io(
    data_section=data_section, memory_efficient=nmb.config['model_config'].get('memory_efficient'),ignore_sources=['config']
)

In [113]:
data_section['app'] 

{'asset': {'info': {'key': 'ZEST_KEY', 'app_date': 'appDate'},
  'table_name': 'app',
  'table_type': 'one_to_one',
  'feature_engineering': 'one_to_one'},
 'data': '#/d/shared/users/jag/penfed/autoindirectv1/nationalmodel3test/lookalike_sample/app_sampled_files.json',
 'io_params': {'drop_duplicates': True,
  'keep_keys': '/d/shared/users/jag/penfed/autoindirectv1/nationalmodel3test/lookalike_sample/keep_keys.json',
  'keep_features': ['appDate',
   'is_client',
   'bisgWhite',
   'bisgBlack',
   'bisgAian',
   'bisgApi',
   'bisgHispanic',
   'bisgMulti',
   'prob_male',
   'prob_female',
   'prob_<62',
   'prob_62+',
   'VS30',
   'VS40',
   'perf_state_cde',
   'perf_acct_typ_cde',
   'kob',
   'flg_exist_fe_trade',
   'ZEST_KEY'],
  'memory_efficient': True}}

## We now see that normalize_io updated the app features to include some of the bisG featurs

### it used this by calling the [schema function](https://github.com/Katlean/model-engine/blob/c78ae3f22054dbc9b08b4c0540318ee0976ec889/model_engine/power/post_sale/base/power_modeling_base.py#L109C81-L109C126) on schema attributed we created (which is default v3 since we didn't set it which grabs the keys from that and turns into list

In [114]:
nmb.schema

'v3'

In [116]:
from model_engine.assets.power.app_asset import schema
schema[nmb.schema]

{'appDate': 'datetime64[ns]',
 'is_client': 'bool',
 'bisgWhite': 'float32',
 'bisgBlack': 'float32',
 'bisgAian': 'float32',
 'bisgApi': 'float32',
 'bisgHispanic': 'float32',
 'bisgMulti': 'float32',
 'prob_male': 'float32',
 'prob_female': 'float32',
 'prob_<62': 'float32',
 'prob_62+': 'float32',
 'VS30': 'float32',
 'VS40': 'float32',
 'perf_state_cde': 'str',
 'perf_acct_typ_cde': 'str',
 'kob': 'str',
 'flg_exist_fe_trade': 'float32',
 'ZEST_KEY': 'str'}

In [57]:
nmb.manifest['data'].keys()

dict_keys(['train_app', 'valid_app', 'test_app', 'train_target', 'valid_target', 'test_target', 'train_fe_data', 'valid_fe_data', 'test_fe_data', 'train_scores', 'valid_scores', 'test_scores', 'train_zest_scores', 'valid_zest_scores', 'test_zest_scores'])

In [58]:
nmb.manifest['data']['train_app']

{'type': 'dataframe',
 'path': 'app/split=train/data=national/data_0.parquet',
 'required': False}

In [59]:
nmb.manifest['data']['train_zest_scores']

{'type': 'dataframe',
 'path': 'overall_zest_scores/split=train/data=national/data_0.parquet',
 'required': False}

In [152]:
data_section['target']

{'asset': {'info': {'col': 'final_DQ60_m24', 'key': 'ZEST_KEY'},
  'table_type': 'target'},
 'data': '#/d/shared/users/jag/penfed/autoindirectv1/nationalmodel3test/lookalike_sample/target_sampled_files.json',
 'io_params': {'drop_duplicates': True,
  'keep_keys': '/d/shared/users/jag/penfed/autoindirectv1/nationalmodel3test/lookalike_sample/keep_keys.json',
  'keep_features': ['ZEST_KEY', 'final_DQ60_m24'],
  'memory_efficient': True}}

## I believe that manifest is created by default and tells the folders you will make with the output

# We then call [make_mb_asset](https://github.com/Katlean/model-engine/blob/c78ae3f22054dbc9b08b4c0540318ee0976ec889/model_engine/power/post_sale/national_model_builder.py#L515C9-L516C1)0

```python
mb_asset = super().make_mb_asset(data_section, model_config)
```
where it is in [power_modeling_base](https://github.com/Katlean/model-engine/blob/38811522d8839d8bc8b547d68c03f336a78f6f8a/model_engine/power/post_sale/base/power_modeling_base.py#L116)

```python
def make_mb_asset(self, data_section, model_config):
    data_section = self.map_target(data_section)
    return {'data': data_section, "config": model_config}

def map_target(self, data_section):
    # Map the target table into exact string "target"
    target_table = [x for x in data_section.keys() if 'target' in x]
    assert len(target_table) == 1, f"Exact one target table is allowed in data_section."
    if target_table[0] != 'target':
        data_section['target'] = data_section.pop(target_table[0])
    return data_section
```

## map_target just makes sure the key for dictioanry with target infromation is named target

## Make mb_asset will create a dictionary with {data: data_section] and config; {model_config

map_target just ensures that the target key in data section is the right name 

Again the data_section contains dictionary which have the locations we will actually grab from to train

In [118]:
nmb.config['model_config']

{'data_split': {'train': {'start_date': '2016-04-01',
   'end_date': '2016-05-01'},
  'valid': {'start_date': '2016-05-01', 'end_date': '2016-06-01'},
  'test': {'start_date': '2016-06-01', 'end_date': '2016-08-01'}},
 'target': 'final_DQ60_m24',
 'base_features': '/d/shared/users/jag/penfed/autoindirectv1/test_only_5_features/keep_features_national_model.json',
 'memory_efficient': True,
 'pipeline_factory': 'BureauPipelineFactory',
 'bivariate_fe_instructions': [],
 'monotonic_constraints_list': [],
 'exclusion_list': [],
 'feature_definition_list': [],
 'key_factor_mapping_list': [],
 'fold_valid': True,
 'mandatory_features': []}

In [40]:
mb_asset = super(NationalModelBuilder, nmb).make_mb_asset(data_section= data_section, model_config= nmb.config['model_config'])

In [120]:
mb_asset.keys()

dict_keys(['data', 'config'])

In [121]:
mb_asset['data'].keys()

dict_keys(['inq', 'app', 'target'])

In [123]:
mb_asset['config']

{'data_split': {'train': {'start_date': '2016-04-01',
   'end_date': '2016-05-01'},
  'valid': {'start_date': '2016-05-01', 'end_date': '2016-06-01'},
  'test': {'start_date': '2016-06-01', 'end_date': '2016-08-01'}},
 'target': 'final_DQ60_m24',
 'base_features': '/d/shared/users/jag/penfed/autoindirectv1/test_only_5_features/keep_features_national_model.json',
 'memory_efficient': True,
 'pipeline_factory': 'BureauPipelineFactory',
 'bivariate_fe_instructions': [],
 'monotonic_constraints_list': [],
 'exclusion_list': [],
 'feature_definition_list': [],
 'key_factor_mapping_list': [],
 'fold_valid': True,
 'mandatory_features': []}

In [64]:
mb_asset['data']['inq']

{'asset': 'equifax/cms_6/inquiry.json',
 'data': '#/d/shared/users/jag/penfed/autoindirectv1/nationalmodel3test/lookalike_sample/inq_sampled_files.json',
 'io_params': {'drop_duplicates': True,
  'keep_keys': '/d/shared/users/jag/penfed/autoindirectv1/nationalmodel3test/lookalike_sample/keep_keys.json',
  'keep_features': ['inq_inquiries__sum'],
  'memory_efficient': True}}

## When you call [`build_model`](https://github.com/Katlean/model-engine/blob/38811522d8839d8bc8b547d68c03f336a78f6f8a/model_engine/model_builder/build_model.py#L26)

```python
def build_model(asset, output_location: str, manifest: dict = None):
    """
    asset: dict
        Model-builder asset.
        See https://zestfinance.atlassian.net/wiki/spaces/BLD/pages/429555720/Model-Builder for more detail.

    output_location: str
        Output path or full S3 URL path (must start with 's3://').

    manifest: dict
        Dict denoting the manifest required by zestio to archive artifacts.
    """
    # Infer which system is used to store output results
    output_system = "s3" if output_location.startswith("s3:") else "dshared"

    print("parsing model-builder asset")
    state = asset_parser(asset)

    data = state["data"]
    config = state.get("config", None) or state.get("asset", None)  # backwards compat
    archive_mode = config.get("archive_mode", "regular")
    fe_version = config.get("fe_version", 1)

    print("Configuring model builder")
    builder = ModelBuilder(
        data=data,
        data_split=config["data_split"],
        archive_mode=archive_mode,
        fe_version=fe_version,
    )
    builder.configure_run(return_outputs=True)

    print("building model")
    artifact_output = builder.generate(input_data=data, input_asset=state["input_asset"], **config)

    # Output the results to different system
    if output_system == "dshared":
        handler = FileSystemHandler()
        prefix = output_location
    else:
        bucket, prefix = global_get_bucket_prefix(output_location)
        handler = S3Handler(bucket_name=bucket)

    data_archiver_object = DataArchiver(
        output_dir=prefix,
        handler=handler,
        manifest=manifest,
        save_manifest=True,
        extras="allow",
    )
    data_archiver_object.archive(artifact_output)

    return builder, artifact_output


In [41]:
output_system = 's3' if supporting_model_path.startswith('s3:') else 'dshared'

In [125]:
output_system

'dshared'

In [42]:
from model_engine.model_builder.asset_parser import asset_parser

In [127]:
mb_asset

{'data': {'inq': {'asset': 'equifax/cms_6/inquiry.json',
   'data': '#/d/shared/users/jag/penfed/autoindirectv1/nationalmodel3test/lookalike_sample/inq_sampled_files.json',
   'io_params': {'drop_duplicates': True,
    'keep_keys': '/d/shared/users/jag/penfed/autoindirectv1/nationalmodel3test/lookalike_sample/keep_keys.json',
    'keep_features': ['inq_inquiries__sum'],
    'memory_efficient': True}},
  'app': {'asset': {'info': {'key': 'ZEST_KEY', 'app_date': 'appDate'},
    'table_name': 'app',
    'table_type': 'one_to_one',
    'feature_engineering': 'one_to_one'},
   'data': '#/d/shared/users/jag/penfed/autoindirectv1/nationalmodel3test/lookalike_sample/app_sampled_files.json',
   'io_params': {'drop_duplicates': True,
    'keep_keys': '/d/shared/users/jag/penfed/autoindirectv1/nationalmodel3test/lookalike_sample/keep_keys.json',
    'keep_features': ['appDate',
     'is_client',
     'bisgWhite',
     'bisgBlack',
     'bisgAian',
     'bisgApi',
     'bisgHispanic',
     'bisgMu

## Asset Parser is in [model_engine/model_builder/asset_parser.py](https://github.com/Katlean/model-engine/blob/c78ae3f22054dbc9b08b4c0540318ee0976ec889/model_engine/model_builder/asset_parser.py#L6C1-L6C25)
```python
def asset_parser(asset):
    asset = copy.deepcopy(asset) # to avoid unwanted changes when executed multiple times
    config = asset.get('config') or asset.get('asset')
    
    state = {'data': {}, 
            'input_asset': copy.deepcopy(asset),
            'config': config,
            'cache': {}
            }

    for name, info in asset['data'].items():
        data, table_name = _load_data(info, config, name)
        state['data'][table_name] = data


    cache = asset.get('cache', {}) or {}
    for name, info in cache.items():
        data, table_name = _load_data(info, config, name)
        state['cache'][table_name] = data

    return state
```

Note that [_load_data](https://github.com/Katlean/model-engine/blob/c78ae3f22054dbc9b08b4c0540318ee0976ec889/model_engine/model_builder/asset_parser.py#L28C1-L28C36) calls dataloader which is in [model_engine/assets/power/spark/sparkscript.py](https://github.com/Katlean/model-engine/blob/c78ae3f22054dbc9b08b4c0540318ee0976ec889/model_engine/assets/power/spark/sparkscript.py#L250C7-L250C80) which calls 
[load_table](https://github.com/Katlean/model-engine/blob/c78ae3f22054dbc9b08b4c0540318ee0976ec889/model_engine/assets/power/spark/sparkscript.py#L198) which calls [load_individual_files](https://github.com/Katlean/model-engine/blob/c78ae3f22054dbc9b08b4c0540318ee0976ec889/model_engine/assets/power/spark/sparkscript.py#L138C9-L138C82)

In [43]:
import copy
parser_asset = copy.deepcopy(mb_asset)

In [387]:
mb_asset['config']

{'data_split': {'train': {'start_date': '2016-04-01',
   'end_date': '2016-05-01'},
  'valid': {'start_date': '2016-05-01', 'end_date': '2016-06-01'},
  'test': {'start_date': '2016-06-01', 'end_date': '2016-08-01'}},
 'target': 'final_DQ60_m24',
 'base_features': '/d/shared/users/jag/penfed/autoindirectv1/test_only_5_features/keep_features_national_model.json',
 'memory_efficient': True,
 'pipeline_factory': 'BureauPipelineFactory',
 'bivariate_fe_instructions': [],
 'monotonic_constraints_list': [],
 'exclusion_list': [],
 'feature_definition_list': [],
 'key_factor_mapping_list': [],
 'fold_valid': True,
 'mandatory_features': []}

In [44]:
parser_config = parser_asset.get('config')

In [155]:
parser_asset['data'].keys()

dict_keys(['inq', 'app', 'target'])

In [130]:
parser_config ## this is just the model_config from the beginning

{'data_split': {'train': {'start_date': '2016-04-01',
   'end_date': '2016-05-01'},
  'valid': {'start_date': '2016-05-01', 'end_date': '2016-06-01'},
  'test': {'start_date': '2016-06-01', 'end_date': '2016-08-01'}},
 'target': 'final_DQ60_m24',
 'base_features': '/d/shared/users/jag/penfed/autoindirectv1/test_only_5_features/keep_features_national_model.json',
 'memory_efficient': True,
 'pipeline_factory': 'BureauPipelineFactory',
 'bivariate_fe_instructions': [],
 'monotonic_constraints_list': [],
 'exclusion_list': [],
 'feature_definition_list': [],
 'key_factor_mapping_list': [],
 'fold_valid': True,
 'mandatory_features': []}

In [45]:
state = {'data': {}, 
            'input_asset': copy.deepcopy(parser_asset),
            'config': parser_config,
            'cache': {}
            }


```python
def _load_data(info, config, name):
    asset_info = info.get('asset', {}) or {}
    io_params = info.get('io_params', {}) or {}
    columns = None
    keep_features, drop_duplicates, keep_keys = _get_io_params(io_params)
    if asset_info:
        asset = load_asset(info['asset'])
        if 'table_name' not in asset.keys():
            asset['table_name'] = name # add table name
        table_name = asset['table_name'] 

        if keep_features:
            if table_name not in ["app", "target", "sample_weight"]:
                asset['info']['keep_features'] = keep_features 

            if asset['info']['key'] not in keep_features:
                columns = keep_features + [asset['info']['key']]
            else:
                columns = keep_features
    else:
        asset = {}
        table_name = name

    data = data_loader(info['data'], asset=asset, columns=columns, drop_duplicates=drop_duplicates, keep_keys=keep_keys)

    return data, table_name
```

In [133]:
 for name, info in parser_asset['data'].items():
     print(f'name: {name} info: {info.keys()}')

name: inq info: dict_keys(['asset', 'data', 'io_params'])
name: app info: dict_keys(['asset', 'data', 'io_params'])
name: target info: dict_keys(['asset', 'data', 'io_params'])


In [80]:
parser_asset['data']['inq']['io_params']

{'drop_duplicates': True,
 'keep_keys': '/d/shared/users/jag/penfed/autoindirectv1/nationalmodel3test/lookalike_sample/keep_keys.json',
 'keep_features': ['inq_inquiries__sum'],
 'memory_efficient': True}

In [81]:
parser_asset['data']['app']['io_params']

{'drop_duplicates': True,
 'keep_keys': '/d/shared/users/jag/penfed/autoindirectv1/nationalmodel3test/lookalike_sample/keep_keys.json',
 'keep_features': ['appDate',
  'is_client',
  'bisgWhite',
  'bisgBlack',
  'bisgAian',
  'bisgApi',
  'bisgHispanic',
  'bisgMulti',
  'prob_male',
  'prob_female',
  'prob_<62',
  'prob_62+',
  'VS30',
  'VS40',
  'perf_state_cde',
  'perf_acct_typ_cde',
  'kob',
  'flg_exist_fe_trade',
  'ZEST_KEY'],
 'memory_efficient': True}

In [148]:
parser_asset['data']['inq']

{'asset': 'equifax/cms_6/inquiry.json',
 'data': '#/d/shared/users/jag/penfed/autoindirectv1/nationalmodel3test/lookalike_sample/inq_sampled_files.json',
 'io_params': {'drop_duplicates': True,
  'keep_keys': '/d/shared/users/jag/penfed/autoindirectv1/nationalmodel3test/lookalike_sample/keep_keys.json',
  'keep_features': ['inq_inquiries__sum'],
  'memory_efficient': True}}

In [134]:
parser_asset['data']['app']['asset']

{'info': {'key': 'ZEST_KEY', 'app_date': 'appDate'},
 'table_name': 'app',
 'table_type': 'one_to_one',
 'feature_engineering': 'one_to_one'}

## Then we basically load the data from [data loader](https://github.com/Katlean/model-engine/blob/c78ae3f22054dbc9b08b4c0540318ee0976ec889/model_engine/io/loaders.py#L27C1-L99C16)

Note if we look at this code, we can basically see it that the info part we feed is the dictionary for that type of source

for example inquiry would be 

{'asset': 'equifax/cms_6/inquiry.json',
 'data': '#/d/shared/users/jag/penfed/autoindirectv1/nationalmodel3test/lookalike_sample/inq_sampled_files.json',
 'io_params': {'drop_duplicates': True,
  'keep_keys': '/d/shared/users/jag/penfed/autoindirectv1/nationalmodel3test/lookalike_sample/keep_keys.json',
  'keep_features': ['inq_inquiries__sum'],
  'memory_efficient': True}}


and [here](https://github.com/Katlean/model-engine/blob/c78ae3f22054dbc9b08b4c0540318ee0976ec889/model_engine/io/loaders.py#L54C17-L54C99 we can see it simply loads the data for each list in the list in that json

```python
data = _load_parquet_dataset(ds.paths, asset, columns, drop_duplicates, keep_keys)
```

In [46]:
from model_engine.model_builder.asset_parser import _load_data

In [47]:
for name, info in parser_asset['data'].items():
    print(f'name, info: {name}, {info}')
    data, table_name = _load_data(info, parser_config, name)
    state['data'][table_name] = data

name, info: inq, {'asset': 'equifax/cms_6/inquiry.json', 'data': '#/d/shared/users/jag/penfed/autoindirectv1/nationalmodel3test/lookalike_sample/inq_sampled_files.json', 'io_params': {'drop_duplicates': True, 'keep_keys': '/d/shared/users/jag/penfed/autoindirectv1/nationalmodel3test/lookalike_sample/keep_keys.json', 'keep_features': ['inq_inquiries__sum'], 'memory_efficient': True}}


name, info: app, {'asset': {'info': {'key': 'ZEST_KEY', 'app_date': 'appDate'}, 'table_name': 'app', 'table_type': 'one_to_one', 'feature_engineering': 'one_to_one'}, 'data': '#/d/shared/users/jag/penfed/autoindirectv1/nationalmodel3test/lookalike_sample/app_sampled_files.json', 'io_params': {'drop_duplicates': True, 'keep_keys': '/d/shared/users/jag/penfed/autoindirectv1/nationalmodel3test/lookalike_sample/keep_keys.json', 'keep_features': ['appDate', 'is_client', 'bisgWhite', 'bisgBlack', 'bisgAian', 'bisgApi', 'bisgHispanic', 'bisgMulti', 'prob_male', 'prob_female', 'prob_<62', 'prob_62+', 'VS30', 'VS40', 'perf_state_cde', 'perf_acct_typ_cde', 'kob', 'flg_exist_fe_trade', 'ZEST_KEY'], 'memory_efficient': True}}


name, info: target, {'asset': {'info': {'col': 'final_DQ60_m24', 'key': 'ZEST_KEY'}, 'table_type': 'target'}, 'data': '#/d/shared/users/jag/penfed/autoindirectv1/nationalmodel3test/lookalike_sample/target_sampled_files.json', 'io_params': {'drop_duplicates': True, 'keep_keys': '/d/shared/users/jag/penfed/autoindirectv1/nationalmodel3test/lookalike_sample/keep_keys.json', 'keep_features': ['ZEST_KEY', 'final_DQ60_m24'], 'memory_efficient': True}}


In [321]:
state['data'].keys()

dict_keys(['inq', 'app', 'target'])

In [322]:
state['data']['inq'].head(5)

,inq_inquiries__sum
ZEST_KEY,
0001533cf06f79a549cec2f2445c0468-e7f306c208e980482006081dff6460b1,7.0
0003aa73530906eb0319fa01702e0713-a0f1e8713294704c9e6e6a7423758216,4.0
0008ecc9f5962036fc715834c5ae9823-8da9126986a298f9c02428d74b66daa0,7.0
0009414d063e6ff229b8bdbdb8b25922-f5959740e2513a5f828de8b41e1aed2a,2.0
0026cf5562b08983938cb8a70f390c3d-4ea2269149b144a7f3ea2c70b32dc63f,1.0


In [323]:
state['data']['app'].head(5)

,appDate,is_client,bisgWhite,bisgBlack,bisgAian,bisgApi,bisgHispanic,bisgMulti,prob_male,prob_female,prob_<62,prob_62+,VS30,VS40,perf_state_cde,perf_acct_typ_cde,kob,flg_exist_fe_trade
ZEST_KEY,,,,,,,,,,,,,,,,,,
93dcaeeee0d24155ca7e790dbb4368f0-6c1d22e62d78404fadb9a7adefda9209,2016-04-15,False,0.876463,0.000231,0.042553,0.000458,0.006527,0.073768,1.0,0.0,0.0,1.0,644.0,638.0,AK,00,FC,0
4eb7803db3f28bfccfc963884fd32e40-0226080a54cbc6966cea48d8f04b16c9,2016-05-27,False,0.448551,0.253056,0.057154,0.088155,0.014718,0.138367,1.0,0.0,1.0,0.0,739.0,756.0,AK,00,FC,0
65e8d67742a0918318ce4accc3c07aef-9edf49e5e7de9aa96695bf6e2fb82d1a,2016-06-28,False,0.932917,0.001938,0.029433,0.002247,0.003293,0.030171,1.0,0.0,1.0,0.0,569.0,506.0,AK,00,FC,0
790ccf92fbb7601281ffc4828cdcdd8a-6244a13fcda04d04c159c9dea9bc2786,2016-05-12,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,555.0,567.0,AK,00,FC,0
c8d3d3080a382b0d6bc4ae55aea08c72-08a18e8af9fcd4bacaa3f830eb70e2ed,2016-05-25,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,591.0,591.0,AK,00,FC,0


In [324]:
state['data']['target'].head(5)

,final_DQ60_m24
ZEST_KEY,
0001533cf06f79a549cec2f2445c0468-e7f306c208e980482006081dff6460b1,0.0
0003aa73530906eb0319fa01702e0713-a0f1e8713294704c9e6e6a7423758216,0.0
0008ecc9f5962036fc715834c5ae9823-8da9126986a298f9c02428d74b66daa0,0.0
0009414d063e6ff229b8bdbdb8b25922-f5959740e2513a5f828de8b41e1aed2a,0.0
0026cf5562b08983938cb8a70f390c3d-4ea2269149b144a7f3ea2c70b32dc63f,0.0


In [48]:
cache = parser_asset.get('cache', {}) or {}
for name, info in parser_asset['data'].items():
    print(f'name, info: {name}, {info}')
    data, table_name = _load_data(info, parser_config, name)
    state['cache'][table_name] = data

name, info: inq, {'asset': 'equifax/cms_6/inquiry.json', 'data': '#/d/shared/users/jag/penfed/autoindirectv1/nationalmodel3test/lookalike_sample/inq_sampled_files.json', 'io_params': {'drop_duplicates': True, 'keep_keys': '/d/shared/users/jag/penfed/autoindirectv1/nationalmodel3test/lookalike_sample/keep_keys.json', 'keep_features': ['inq_inquiries__sum'], 'memory_efficient': True}}


name, info: app, {'asset': {'info': {'key': 'ZEST_KEY', 'app_date': 'appDate'}, 'table_name': 'app', 'table_type': 'one_to_one', 'feature_engineering': 'one_to_one'}, 'data': '#/d/shared/users/jag/penfed/autoindirectv1/nationalmodel3test/lookalike_sample/app_sampled_files.json', 'io_params': {'drop_duplicates': True, 'keep_keys': '/d/shared/users/jag/penfed/autoindirectv1/nationalmodel3test/lookalike_sample/keep_keys.json', 'keep_features': ['appDate', 'is_client', 'bisgWhite', 'bisgBlack', 'bisgAian', 'bisgApi', 'bisgHispanic', 'bisgMulti', 'prob_male', 'prob_female', 'prob_<62', 'prob_62+', 'VS30', 'VS40', 'perf_state_cde', 'perf_acct_typ_cde', 'kob', 'flg_exist_fe_trade', 'ZEST_KEY'], 'memory_efficient': True}}


name, info: target, {'asset': {'info': {'col': 'final_DQ60_m24', 'key': 'ZEST_KEY'}, 'table_type': 'target', 'table_name': 'target'}, 'data': '#/d/shared/users/jag/penfed/autoindirectv1/nationalmodel3test/lookalike_sample/target_sampled_files.json', 'io_params': {'drop_duplicates': True, 'keep_keys': '/d/shared/users/jag/penfed/autoindirectv1/nationalmodel3test/lookalike_sample/keep_keys.json', 'keep_features': ['ZEST_KEY', 'final_DQ60_m24'], 'memory_efficient': True}}


# We now have our state dictionary with all of the files actually loaded

In [56]:
from modelengine.model_engine import ModelBuilder

ModuleNotFoundError: No module named 'modelengine'

In [49]:
data = state['data']

In [50]:
state.keys()

dict_keys(['data', 'input_asset', 'config', 'cache'])

In [51]:
def create_builder_object(data, data_split, archive_mode):
    builder = ModelBuilder(data=data, data_split=data_split, archive_mode=archive_mode)
    builder.configure_run(return_outputs=True)
    return builder

In [52]:
builder = create_builder_object(data = state['data'], data_split = state['config']['data_split'], archive_mode='regular')

NameError: name 'ModelBuilder' is not defined

In [ ]:
config_for_builder_generate = state['config']

In [ ]:
first_node = builder.graph.nodes[0]

In [ ]:
optional_artifacts_set = builder._ArtifactManager__get_optional_artifacts_set()

# Initialize the artifacts pool with your inputs
builder.artifacts_pool = {}

inputs_for_builder_generate = {
    **state['config'],                           # Unpacks everything from your config dict
    'input_data': state['data'],                 # Adds input_data
    'input_asset': state['input_asset'] # Adds input_asset
}

builder._ArtifactManager__feed_pool(inputs_for_builder_generate, builder.artifacts_pool)
builder._ArtifactManager__reset_is_run_dict()


In [393]:
state['config']

{'data_split': {'train': {'start_date': '2016-04-01',
   'end_date': '2016-05-01'},
  'valid': {'start_date': '2016-05-01', 'end_date': '2016-06-01'},
  'test': {'start_date': '2016-06-01', 'end_date': '2016-08-01'}},
 'target': 'final_DQ60_m24',
 'base_features': '/d/shared/users/jag/penfed/autoindirectv1/test_only_5_features/keep_features_national_model.json',
 'memory_efficient': True,
 'pipeline_factory': 'BureauPipelineFactory',
 'bivariate_fe_instructions': [],
 'monotonic_constraints_list': [],
 'exclusion_list': [],
 'feature_definition_list': [],
 'key_factor_mapping_list': [],
 'fold_valid': True,
 'mandatory_features': []}

In [396]:
state['config'].keys()

dict_keys(['data_split', 'target', 'base_features', 'memory_efficient', 'pipeline_factory', 'bivariate_fe_instructions', 'monotonic_constraints_list', 'exclusion_list', 'feature_definition_list', 'key_factor_mapping_list', 'fold_valid', 'mandatory_features'])

In [394]:
state['data'].keys()

dict_keys(['inq', 'app', 'target'])

In [395]:
state['input_asset'].keys()

dict_keys(['data', 'config'])

In [397]:
state['input_asset']['config'].keys()

dict_keys(['data_split', 'target', 'base_features', 'memory_efficient', 'pipeline_factory', 'bivariate_fe_instructions', 'monotonic_constraints_list', 'exclusion_list', 'feature_definition_list', 'key_factor_mapping_list', 'fold_valid', 'mandatory_features'])

# When we call  generate we feed in all the dictionarys

## State['config'] this is simply the model_config. Going all the way back it is config['model_config'] but updated with the pipeline
## state['data'] this is dictionary which has the data set for each key for all the tables we will load in 
## state['input_asset'] this is the mb_asset we created and put into state

# **Why do we need this model config twice????????? it is in the mb asset and the data

In [385]:
builder.artifacts_pool.keys() ==inputs_for_builder_generate.keys()

True

In [398]:
node_input_names = first_node.input_names
node_inputs = {k: v for k, v in builder.artifacts_pool.items() if k in node_input_names}  # grab inputs from pool

In [399]:
builder.artifacts_pool.keys()

dict_keys(['data_split', 'target', 'base_features', 'memory_efficient', 'pipeline_factory', 'bivariate_fe_instructions', 'monotonic_constraints_list', 'exclusion_list', 'feature_definition_list', 'key_factor_mapping_list', 'fold_valid', 'mandatory_features', 'input_data', 'input_asset'])

In [438]:
type(first_node), first_node

(zaml.artifact_engine.artifacts.base.InputArtifact,
 <Artifact name: 'input_asset', class name: 'InputArtifact', inputs: ['input_asset']>)

In [447]:
first_node.__dict__

{'name': 'input_asset',
 'required_input_names': ['input_asset'],
 'input_names': ['input_asset'],
 'shared_params': {},
 '_zaml_ver': '34.4.1',
 '_other_params': {},
 'inbounds': [],
 'outbounds': [<Artifact name: 'asset', class name: 'InputAssetArtifact', inputs: ['input_asset']>,
  <Artifact name: 'static_asset', class name: 'StaticAssetArtifact', inputs: ['input_asset', 'pipeline', 'pipe_factory']>]}

## This first node is an input artifact: so we cacall its own [generate](https://github.com/Katlean/zaml/blob/e4c61a6d16b70ed4e21b7704a936eb624d9dd318/zaml/artifact_engine/artifacts/base.py#L43C1-L51C33) method 

```python
class InputArtifact(BaseArtifact):
    def __init__(self, name: str = None):
        self.name = name
        self.required_input_names = [name]

        super().__init__()

    def generate(self, **inputs):
        return inputs[self.name]

```

In [431]:
artf_first_node = builder._generate_single_node(first_node, optional_artifacts_set)

In [415]:
node_input_names = first_node.input_names
node_inputs = {k: v for k, v in builder.artifacts_pool.items() if k in node_input_names}  # grab inputs from pool

generated_output = first_node.generate(**node_inputs)


In [433]:
artf_first_node== generated_output

True

In [451]:
first_node.name

'input_asset'

In [ ]:
import logging
import pandas as pd
from time import time
from datetime import timedelta

# We need to detect _NoOutput. If you can't import the class directly, 
# we check the type name.
def is_no_output(obj):
    return type(obj).__name__ == '_NoOutput'

def generate_with_capture(builder, optional_artifacts_set):
    # 1. Suppress ZAML logging
    logger = logging.getLogger('zaml')
    original_level = logger.level
    logger.setLevel(logging.CRITICAL)
    
    art_dict = {}
    log_buffer = []
    
    def log(msg=""):
        log_buffer.append(str(msg))
    
    # Ensure the pool exists
    if not hasattr(builder, 'artifacts_pool'):
        builder.artifacts_pool = {}
    
    log(f"Starting execution for {len(builder.graph.nodes)} nodes...\n")
    
    try:
        for i, node in enumerate(builder.graph.nodes):
            start_time = time()  # Start timing
            
            name = node.name
            log(f'[{i}] NODE: {name}')
            log(f'    Type: {type(node).__name__}')
            
            # Context Printing Logic
            node_input_names = node.input_names
            node_inputs = {k: v for k, v in builder.artifacts_pool.items() if k in node_input_names}
            
            log(f'    Required Inputs: {node_input_names}')
            if not node_inputs:
                log(f'    Input Contexts: [None / No inputs available yet]')
            else:
                log(f'    Input Contexts:')
                for input_name, input_value in node_inputs.items():
                    if hasattr(input_value, 'shape'):
                        desc = f"<{type(input_value).__name__} shape={input_value.shape}>"
                    elif isinstance(input_value, dict):
                        desc = f"<dict with keys: {list(input_value.keys())}>"
                    else:
                        desc = str(input_value)
                        if len(desc) > 200: 
                            desc = desc[:200] + "..."
                    log(f'        - {input_name}: {desc}')
            
            # --- Execution ---
            try:
                artf = builder._generate_single_node(node, optional_artifacts_set)
                
                # --- REQUESTED LOGIC START ---
                
                # 1. Archive and Update Pool if not _NoOutput
                if not is_no_output(artf):
                    builder.artifacts_pool[node.name] = artf
                    art_dict[node.name] = artf
                    
                    # Call the internal archive method
                    builder._archive_single_node(node, artf)
                    
                    log(f'    --> Generated Output: Success (Archived)')
                else:
                    log(f'    --> Generated Output: SKIPPED (NoOutput)')
                
                # 2. Clear Pool if cache is False
                if not builder.cache:
                    # Accessing private method via name mangling
                    if hasattr(builder, '_ArtifactManager__clear_pool'):
                        builder._ArtifactManager__clear_pool(node, builder.artifacts_pool)
                    else:
                        # Fallback if method name is different in runtime environment
                        pass
                
                # 3. Time Tracking
                if node not in builder.input_artifacts:
                    run_time = time() - start_time
                    log(f'    --> Time Spent: {timedelta(seconds=run_time)}\n')
                else:
                    log(f'\n')  # Just a newline for inputs
                
                # --- REQUESTED LOGIC END ---
            except Exception as e:
                log(f'    --> Generated Output: ERROR (Ignored) - {e}\n')
    finally:
        logger.setLevel(original_level)
    
    return builder, art_dict, "\n".join(log_buffer)
import logging
import pandas as pd

def generate_with_print(builder, optional_artifacts_set):
    # 1. Suppress ZAML logging to "ignore msgs"
    logger = logging.getLogger('zaml')
    original_level = logger.level
    logger.setLevel(logging.CRITICAL)

    art_dict = {}

    # Ensure the pool exists
    if not hasattr(builder, 'artifacts_pool'):
        builder.artifacts_pool = {}

    print(f"Starting execution for {len(builder.graph.nodes)} nodes...\n")

    try:
        for i, node in enumerate(builder.graph.nodes):
            name = node.name
            
            print(f'[{i}] NODE: {name}')
            print(f'    Type: {type(node).__name__}')
            
            # --- FIXED: Use current node's input names ---
            node_input_names = node.input_names
            
            # Grab inputs from the builder's pool
            node_inputs = {k: v for k, v in builder.artifacts_pool.items() if k in node_input_names}
            
            # --- REQUESTED: Print Input Contexts ---
            print(f'    Required Inputs: {node_input_names}')
            if not node_inputs:
                print(f'    Input Contexts: [None / No inputs available yet]')
            else:
                print(f'    Input Contexts:')
                for input_name, input_value in node_inputs.items():
                    # Format the output to be readable
                    if hasattr(input_value, 'shape'): # Pandas objects
                        desc = f"<{type(input_value).__name__} shape={input_value.shape}>"
                    elif isinstance(input_value, dict): # Dictionaries
                        desc = f"<dict with keys: {list(input_value.keys())}>"
                    else: # Everything else (strings, ints, simple objects)
                        desc = str(input_value)
                        # Truncate if it's massive
                        if len(desc) > 200: 
                            desc = desc[:200] + "..."
                    
                    print(f'        - {input_name}: {desc}')

            # --- Execution ---
            try:
                # Generate the node
                art = builder._generate_single_node(node, optional_artifacts_set)
                
                # Update the pool so the NEXT node can find this output
                # (ZAML returns _NoOutput class for skipped nodes, so we check type)
                if type(art).__name__ != '_NoOutput':
                    builder.artifacts_pool[name] = art
                    art_dict[name] = art
                    print(f'    --> Generated Output: Success\n')
                else:
                    print(f'    --> Generated Output: SKIPPED (Optional/Missing Inputs)\n')

            except Exception as e:
                # "Ignore errors" behavior
                print(f'    --> Generated Output: ERROR (Ignored) - {e}\n')

    finally:
        # Restore original logging level
        logger.setLevel(original_level)
    ## we are b
    return builder, art_dict

In [ ]:
# After we generate the artifact with gnerate_single_node (where we call the generate of that type of artifact with 
# give that artifact the value in the dctionary artifacts pool (if it is not a noOutput Artifact)



In [ ]:
builder_after_running, art_dict, output_txt = generate_with_capture(builder, optional_artifacts_set)

In [485]:
builder_after_running.artifacts_pool.keys()

dict_keys(['base_features', 'memory_efficient', 'feature_definition_list', 'key_factor_mapping_list', 'fold_valid', 'mandatory_features', 'input_asset', 'train_sample_weight', 'valid_sample_weight', 'test_sample_weight', 'neutralization_list', 'n_tiers', 'value_based_key_factor_mapping_list', 'parsed_monotonic_constraints_list', 'data', 'pipe_factory_config', 'parsed_exclusion_list', 'parsed_biv_fe_instructions', 'fit_params', 'data_dict', 'cat_map', 'train_target', 'train_app', 'train_data', 'valid_target', 'valid_app', 'valid_data', 'test_target', 'test_app', 'test_data'])

In [488]:
builder_after_running.artifacts_pool['base_features']

'/d/shared/users/jag/penfed/autoindirectv1/test_only_5_features/keep_features_national_model.json'

In [489]:
builder_after_running.artifacts_pool['key_factor_mapping_list']

[]

In [490]:
builder_after_running.artifacts_pool['fold_valid']

True

In [491]:
builder_after_running.artifacts_pool['fit_params']

{}

In [492]:
builder_after_running.artifacts_pool['parsed_monotonic_constraints_list']

{'trade_months_since_openDate__max': -1,
 'trade_months_since_openDate__max_by_acctType_auto': -1,
 'trade_months_since_openDate__max_by_acctType_charge_card': -1,
 'trade_months_since_openDate__max_by_acctType_credit_card': -1,
 'trade_months_since_openDate__max_by_acctType_education': -1,
 'trade_months_since_openDate__max_by_acctType_secure': -1,
 'trade_months_since_openDate__max_by_acctType_unsecure': -1,
 'trade_months_since_openDate__max_by_acctType_utility': -1,
 'trade_months_since_openDate__max_by_ecoa_authorized': -1,
 'trade_months_since_openDate__max_by_ecoa_individual': -1,
 'trade_months_since_openDate__max_by_ecoa_joint': -1,
 'trade_months_since_openDate__max_by_prtfType_installment': -1,
 'trade_months_since_openDate__max_by_prtfType_mortgage': -1,
 'trade_months_since_openDate__max_by_prtfType_revolving': -1,
 'trade_months_since_openDate__mean': -1,
 'trade_months_since_openDate__mean_by_acctType_auto': -1,
 'trade_months_since_openDate__mean_by_acctType_charge_card

In [ ]:
builder_after_running.artifacts_pool['parsed_monotonic_constraints_list']

In [482]:

file_name = 'generate_output_list.txt'


In [483]:
with open(file_name, 'w') as f:
    f.write(output_txt)
print(f"Successfully wrote output to: {full_path}")

Successfully wrote output to: client-project-penfed/autoindirect/modeling/custom_model/psk/generate_output_list.txt


In [57]:
from model_engine.model_builder import ModelBuilder

In [58]:
builder = create_builder_object(data = state['data'], data_split = state['config']['data_split'], archive_mode='regular')

In [116]:
data['inq']

,inq_inquiries__sum
ZEST_KEY,
0001533cf06f79a549cec2f2445c0468-e7f306c208e980482006081dff6460b1,7.0
0003aa73530906eb0319fa01702e0713-a0f1e8713294704c9e6e6a7423758216,4.0
0008ecc9f5962036fc715834c5ae9823-8da9126986a298f9c02428d74b66daa0,7.0
0009414d063e6ff229b8bdbdb8b25922-f5959740e2513a5f828de8b41e1aed2a,2.0
0026cf5562b08983938cb8a70f390c3d-4ea2269149b144a7f3ea2c70b32dc63f,1.0
...,...
9974134_9453852615,2.0
9974492_9422873173,1.0
9985272_9422721360,1.0


In [117]:
data['app']

,appDate,is_client,bisgWhite,bisgBlack,bisgAian,bisgApi,bisgHispanic,bisgMulti,prob_male,prob_female,prob_<62,prob_62+,VS30,VS40,perf_state_cde,perf_acct_typ_cde,kob,flg_exist_fe_trade
ZEST_KEY,,,,,,,,,,,,,,,,,,
93dcaeeee0d24155ca7e790dbb4368f0-6c1d22e62d78404fadb9a7adefda9209,2016-04-15,False,0.876463,0.000231,0.042553,0.000458,0.006527,0.073768,1.000000,0.000000,0.0,1.0,644.0,638.0,AK,00,FC,0
4eb7803db3f28bfccfc963884fd32e40-0226080a54cbc6966cea48d8f04b16c9,2016-05-27,False,0.448551,0.253056,0.057154,0.088155,0.014718,0.138367,1.000000,0.000000,1.0,0.0,739.0,756.0,AK,00,FC,0
65e8d67742a0918318ce4accc3c07aef-9edf49e5e7de9aa96695bf6e2fb82d1a,2016-06-28,False,0.932917,0.001938,0.029433,0.002247,0.003293,0.030171,1.000000,0.000000,1.0,0.0,569.0,506.0,AK,00,FC,0
790ccf92fbb7601281ffc4828cdcdd8a-6244a13fcda04d04c159c9dea9bc2786,2016-05-12,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,555.0,567.0,AK,00,FC,0
c8d3d3080a382b0d6bc4ae55aea08c72-08a18e8af9fcd4bacaa3f830eb70e2ed,2016-05-25,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,591.0,591.0,AK,00,FC,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14944474_9434284748,2016-07-11,False,0.980519,0.000765,0.000000,0.001670,0.008063,0.008983,0.996294,0.003706,1.0,0.0,640.0,663.0,WY,AU,QU,1
11506371_10198447126,2016-07-11,False,0.981447,0.000000,0.013984,0.000172,0.002099,0.002298,0.995169,0.004831,1.0,0.0,502.0,569.0,WY,AU,BB,0
21957968_9428081087,2016-07-05,False,0.979871,0.000000,0.007403,0.000172,0.002840,0.009714,0.992999,0.007001,1.0,0.0,683.0,733.0,WY,AU,QU,1


In [59]:
full_artifact = builder.generate(input_data=data, input_asset=state['input_asset'], **state['config'])

INFO:zaml.artifact_engine.logger:Executing InputArtifact <input_asset>...
INFO:zaml.artifact_engine.logger:Executing InputArtifact <input_data>...
INFO:zaml.artifact_engine.logger:Executing InputArtifact <monotonic_constraints_list>...
INFO:zaml.artifact_engine.logger:Executing InputArtifact <data_split>...
INFO:zaml.artifact_engine.logger:Executing InputArtifact <train_sample_weight>...
INFO:zaml.artifact_engine.logger:Not all required inputs are available for optional artifact <train_sample_weight>, thus it will be omitted.
INFO:zaml.artifact_engine.logger:Executing InputArtifact <valid_sample_weight>...
INFO:zaml.artifact_engine.logger:Not all required inputs are available for optional artifact <valid_sample_weight>, thus it will be omitted.
INFO:zaml.artifact_engine.logger:Executing InputArtifact <test_sample_weight>...
INFO:zaml.artifact_engine.logger:Not all required inputs are available for optional artifact <test_sample_weight>, thus it will be omitted.
INFO:zaml.artifact_engin

-------------------------
Name: app
Transformer type: None
Number of features: 18
Time spent: 0.044s
-------------------------
Name: inq
Transformer type: None
Number of features: 1
Time spent: 0.020s
-------------------------
Name: app FE
Transformer type: OneToOneEngine
Number of features: 0
Time spent: 0.096s
-------------------------
Name: inq FE
Transformer type: InquiryFeatureEngine
Number of features: 1
Time spent: 0.061s


-------------------------
Name: Merge data
Transformer type: Concat
Number of features: 1
Time spent: 3.252s
-------------------------
Name: FillNA
Transformer type: FillNA
Number of features: 1
Time spent: 0.031s
-------------------------
Name: GeneralFeatureSelection
Transformer type: GeneralFeatureSelection
Number of features: 1
Time spent: 0.002s
[0]	train-auc:0.62610	val-auc:0.62682
[100]	train-auc:0.62610	val-auc:0.62682
[200]	train-auc:0.62610	val-auc:0.62682
[208]	train-auc:0.62610	val-auc:0.62682


INFO:zaml.artifact_engine.logger:Finished <pipeline_fitter>, total time spent: 0:00:13.490987
INFO:zaml.artifact_engine.logger:Executing FittedPipeline <pipeline>...
INFO:zaml.artifact_engine.logger:Finished <pipeline>, total time spent: 0:00:00.000506
INFO:zaml.artifact_engine.logger:Executing FitTimeInfoArtifact <fit_time_info>...
INFO:zaml.artifact_engine.logger:Finished <fit_time_info>, total time spent: 0:00:00.000454
INFO:zaml.artifact_engine.logger:Executing PipeFactoryArtifact <pipe_factory>...
INFO:zaml.artifact_engine.logger:Finished <pipe_factory>, total time spent: 0:00:00.000464
INFO:zaml.artifact_engine.logger:Executing FittedModel <model>...
INFO:zaml.artifact_engine.logger:Finished <model>, total time spent: 0:00:00.000592
INFO:zaml.artifact_engine.logger:Executing FeDataArtifact <train_fe_data>...


-------------------------
Name: app
Transformer type: None
Number of features: 18
Time spent: 0.000s
-------------------------
Name: inq
Transformer type: None
Number of features: 1
Time spent: 0.000s
-------------------------
Name: app FE
Transformer type: OneToOneEngine
Number of features: 0
Time spent: 0.048s
-------------------------
Name: inq FE
Transformer type: InquiryFeatureEngine
Number of features: 1
Time spent: 0.022s


...


INFO:zaml.artifact_engine.logger:Finished <valid_fe_data>, total time spent: 0:00:01.648571
INFO:zaml.artifact_engine.logger:Executing FeDataArtifact <test_fe_data>...


-------------------------
Name: Merge data
Transformer type: Concat
Number of features: 1
Time spent: 1.564s
-------------------------
Name: FillNA
Transformer type: FillNA
Number of features: 1
Time spent: 0.015s
-------------------------
Name: GeneralFeatureSelection
Transformer type: GeneralFeatureSelection
Number of features: 1
Time spent: 0.001s
-------------------------
Name: app
Transformer type: None
Number of features: 18
Time spent: 0.000s
-------------------------
Name: inq
Transformer type: None
Number of features: 1
Time spent: 0.000s
-------------------------
Name: app FE
Transformer type: OneToOneEngine
Number of features: 0
Time spent: 0.078s
-------------------------
Name: inq FE
Transformer type: InquiryFeatureEngine
Number of features: 1
Time spent: 0.023s



INFO:zaml.artifact_engine.logger:Finished <test_fe_data>, total time spent: 0:00:03.468363
INFO:zaml.artifact_engine.logger:Executing StaticAssetArtifact <static_asset>...
INFO:zaml.artifact_engine.logger:Finished <static_asset>, total time spent: 0:00:00.017162
INFO:zaml.artifact_engine.logger:Executing TrainHistoryArtifact <train_history>...
INFO:zaml.artifact_engine.logger:Finished <train_history>, total time spent: 0:00:00.000546
INFO:zaml.artifact_engine.logger:Executing BestModelParamsArtifact <best_model_params>...
INFO:zaml.artifact_engine.logger:Finished <best_model_params>, total time spent: 0:00:00.000394
INFO:zaml.artifact_engine.logger:Executing ScoresArtifact <train_scores>...
INFO:zaml.artifact_engine.logger:Finished <train_scores>, total time spent: 0:00:00.041299
INFO:zaml.artifact_engine.logger:Executing CalibrationObjectArtifact <calibration_object>...


-------------------------
Name: Merge data
Transformer type: Concat
Number of features: 1
Time spent: 3.334s
-------------------------
Name: FillNA
Transformer type: FillNA
Number of features: 1
Time spent: 0.018s
-------------------------
Name: GeneralFeatureSelection
Transformer type: GeneralFeatureSelection
Number of features: 1
Time spent: 0.001s


INFO:zaml.artifact_engine.logger:Finished <calibration_object>, total time spent: 0:00:15.100141
INFO:zaml.artifact_engine.logger:Executing SubmodelScoresArtifact <train_submodel_scores>...
INFO:zaml.artifact_engine.logger:Finished <train_submodel_scores>, total time spent: 0:00:00.000938
INFO:zaml.artifact_engine.logger:Executing ScoresArtifact <valid_scores>...
INFO:zaml.artifact_engine.logger:Finished <valid_scores>, total time spent: 0:00:00.060012
INFO:zaml.artifact_engine.logger:Executing SubmodelScoresArtifact <valid_submodel_scores>...
INFO:zaml.artifact_engine.logger:Finished <valid_submodel_scores>, total time spent: 0:00:00.000987
INFO:zaml.artifact_engine.logger:Executing ScoresArtifact <test_scores>...
INFO:zaml.artifact_engine.logger:Finished <test_scores>, total time spent: 0:00:00.052183
INFO:zaml.artifact_engine.logger:Executing SubmodelScoresArtifact <test_submodel_scores>...
INFO:zaml.artifact_engine.logger:Finished <test_submodel_scores>, total time spent: 0:00:00.0

In [102]:
full_artifact.keys()

dict_keys(['asset', 'versions', 'parsed_monotonic_constraints_list', 'splitter', 'train_target', 'train_app', 'valid_target', 'valid_app', 'test_target', 'test_app', 'train_data_summary', 'valid_data_summary', 'test_data_summary', 'pipeline', 'fit_time_info', 'model', 'train_fe_data', 'valid_fe_data', 'test_fe_data', 'static_asset', 'train_history', 'best_model_params', 'train_scores', 'calibration_object', 'valid_scores', 'test_scores', 'feature_definition', 'key_factors_mapping', 'value_based_key_factor_mapping', 'feature_importance', 'model_strategy', 'mrm_pipeline', 'score_recalibration_mapping', 'train_auc', 'train_ks', 'valid_auc', 'valid_ks', 'test_auc', 'test_ks', 'top_features', 'keep_features', 'train_zest_scores', 'valid_zest_scores', 'test_zest_scores'])

In [128]:
full_artifact.keys()

dict_keys(['asset', 'versions', 'parsed_monotonic_constraints_list', 'splitter', 'train_target', 'train_app', 'valid_target', 'valid_app', 'test_target', 'test_app', 'train_data_summary', 'valid_data_summary', 'test_data_summary', 'pipeline', 'fit_time_info', 'model', 'train_fe_data', 'valid_fe_data', 'test_fe_data', 'static_asset', 'train_history', 'best_model_params', 'train_scores', 'calibration_object', 'valid_scores', 'test_scores', 'feature_definition', 'key_factors_mapping', 'value_based_key_factor_mapping', 'feature_importance', 'model_strategy', 'mrm_pipeline', 'score_recalibration_mapping', 'train_auc', 'train_ks', 'valid_auc', 'valid_ks', 'test_auc', 'test_ks', 'top_features', 'keep_features', 'train_zest_scores', 'valid_zest_scores', 'test_zest_scores'])

In [138]:
full_artifact['best_model_params']

{'best_score': 0.6268342834597741, 'best_iteration': 9, 'best_ntree_limit': 10}

In [104]:
full_artifact['pipeline'].__dict__

{'_zaml_ver': '34.4.1',
 '_other_params': {},
 '_dep_vers': {'dill': '0.3.6',
  'docmill': '0.1.2',
  'feather-format': '0.4.1',
  'graphviz': '0.21',
  'h5py': '3.15.1',
  'hyperopt': '0.2.7',
  'ipython': '8.30.0',
  'ipywidgets': '8.1.8',
  'joblib': '1.5.3',
  'jupyter-book': '2.1.0',
  'jupyter_server_mathjax': '0.2.6',
  'kaleido': '1.2.0',
  'lightgbm': '3.3.5',
  'matplotlib': '3.10.8',
  'numba': '0.56.4',
  'numpy': '1.23.5',
  'openpyxl': '3.1.5',
  'pandas': '2.0.2',
  'pg8000': '1.31.5',
  'plotly': '5.20.0',
  'pyarrow': '11.0.0',
  'pyarrow_hotfix': '0.7',
  'PypeR': '1.1.2',
  'PyYAML': '6.0.3',
  'schedule': '1.2.2',
  'scikit-learn': '1.2.2',
  'scipy': '1.10.1',
  'seaborn': '0.13.2',
  'shap': '0.41.0',
  'simplejson': '3.20.2',
  'Sphinx': '8.0.2',
  'sphinx-rtd-theme': '3.1.0',
  'sphinxcontrib-applehelp': '1.0.7',
  'sphinxcontrib-bibtex': '2.6.1',
  'sphinxcontrib-devhelp': '1.0.5',
  'sphinxcontrib-htmlhelp': '2.0.4',
  'sphinxcontrib-qthelp': '1.0.6',
  'sphin

In [106]:
type(full_artifact['pipeline'].init_transformers)

list

In [129]:
len(full_artifact['pipeline'].init_transformers)-1

4

In [132]:
full_artifact['pipeline'].init_transformers[4].transformer

In [134]:
for i in range(len(full_artifact['pipeline'].init_transformers)):
    print(f'\n')
    print(type(full_artifact['pipeline'].init_transformers[i]), full_artifact['pipeline'].init_transformers[i].__dict__)
    print(f'\n')



<class 'zaml.model.pipeline.pipeline.InputNode'> {'_zaml_ver': '34.4.1', '_other_params': {}, 'name': 'app', 'inbounds': [], 'outbounds': ['app FE'], 'transformer': None, 'data': None, 'valid_data': None, 'feature_types': None, 'data_cached': None, 'fit_params': None, 'is_run': False}




<class 'zaml.model.pipeline.pipeline.InputNode'> {'_zaml_ver': '34.4.1', '_other_params': {}, 'name': 'inq', 'inbounds': [], 'outbounds': ['inq FE'], 'transformer': None, 'data': None, 'valid_data': None, 'feature_types': None, 'data_cached': None, 'fit_params': None, 'is_run': False}




<class 'zaml.model.pipeline.pipeline.TransformNode'> {'_zaml_ver': '34.4.1', '_other_params': {}, 'name': 'Merge data', 'inbounds': ['app FE', 'inq FE'], 'outbounds': [], 'transformer': <feature_engine_parts.transformers.joins.Concat object at 0x7f63e172d240>, 'data': None, 'valid_data': None, 'feature_types': None, 'data_cached': None, 'fit_params': None, 'is_run': False}




<class 'zaml.model.pipeline.pipeline.T

In [136]:
full_artifact['pipeline'].model

In [137]:
full_artifact['pipeline'].model.__dict__

{'_zaml_ver': '34.4.1',
 '_other_params': {'eval_metric': 'auc', 'tree_method': 'hist'},
 'model_name': 'xgboost_model_ffaadb60-e65e-4637-a145-b80f814cda71',
 'model': <xgboost.core.Booster at 0x7f65b23cfdf0>,
 'feature_types': [('inq_inquiries__sum', 'float32')],
 'enable_index_matching': True,
 'explainer_type': 'TreeExplainer',
 'max_depth': 6,
 'learning_rate': 0.02,
 'n_estimators': 25000,
 'early_stopping_rounds': 200,
 'n_min_estimators': 0,
 'objective': 'binary:logistic',
 'booster': 'gbtree',
 'gamma': 0,
 'min_child_weight': 150,
 'max_delta_step': 0,
 'subsample': 0.9,
 'colsample_bytree': 0.25,
 'colsample_bylevel': 1,
 'reg_alpha': 0,
 'reg_lambda': 5,
 'scale_pos_weight': 4,
 'max_leaves': 0,
 'base_score': 0.5,
 'seed': 12,
 'monotone_constraints': {'trade_months_since_openDate__max': -1,
  'trade_months_since_openDate__max_by_acctType_auto': -1,
  'trade_months_since_openDate__max_by_acctType_charge_card': -1,
  'trade_months_since_openDate__max_by_acctType_credit_card

In [ ]:
full_artifact['pipeline'].model.model

In [98]:
import inspect

print(inspect.getsource(builder.archivers[0].save_manifest))

AttributeError: 'DictionaryOutput' object has no attribute 'save_manifest'

In [94]:
len(builder.archivers)

1

In [87]:
type(full_artifact)

dict

In [60]:
from zestio import DataArchiver

In [61]:
from zestio.handler import FileSystemHandler, S3Handler

In [62]:
handler = FileSystemHandler()
prefix = supporting_model_path

In [63]:
data_archiver_object = DataArchiver(
    output_dir=prefix, handler=handler, manifest=nmb.manifest, save_manifest=True, extras='allow'
)

In [64]:
data_archiver_object.__dict__.keys()

dict_keys(['handler', 'output_dir', 'artf_manifest', 'save_manifest', 'table_format', 'extras'])

In [65]:
data_archiver_object.archive(full_artifact)

In [66]:
supporting_model_path

'/d/shared/users/jag/penfed/autoindirectv1/nationalmodel3test/supporting_model'

In [67]:
ls /d/shared/users/jag/penfed/autoindirectv1/nationalmodel3test/supporting_model

app/                                    score_recalibration_mapping.json
artifact_manifest.json                  splitter.obj
asset.json                              static_asset.json
best_model_params.json                  target/
calibration_object.obj                  test_auc.json
feature_definition.parquet              test_data_summary.json
feature_importance.parquet              test_ks.json
fe_data/                                top_features.parquet
fit_time_info.json                      train_auc.json
keep_features.json                      train_data_summary.json
key_factors_mapping.json                train_history.json
model.obj                               train_ks.json
model_strategy.json                     valid_auc.json
mrm_pipeline.obj                        valid_data_summary.json
overall_scores/                         valid_ks.json
overall_zest_scores/                    value_based_key_factor_mapping.json
parsed_monotonic_constraints_list.json  versions.json
pi

In [ ]:
## NOW NEED TO CONFIRM IF WE TRAINED A FINAL MODEL IT WOULD 

In [68]:
nmb._add_data_description(supporting_model_path, supporting_model_path, 'data_description.json')

In [69]:
ls /d/shared/users/jag/penfed/autoindirectv1/nationalmodel3test/supporting_model

app/                                    pipeline.obj
artifact_manifest.json                  score_recalibration_mapping.json
asset.json                              splitter.obj
best_model_params.json                  static_asset.json
calibration_object.obj                  target/
data_description.json                   test_auc.json
feature_definition.parquet              test_data_summary.json
feature_importance.parquet              test_ks.json
fe_data/                                top_features.parquet
fit_time_info.json                      train_auc.json
keep_features.json                      train_data_summary.json
key_factors_mapping.json                train_history.json
model.obj                               train_ks.json
model_strategy.json                     valid_auc.json
mrm_pipeline.obj                        valid_data_summary.json
overall_scores/                         valid_ks.json
overall_zest_scores/                    value_based_key_factor_mapping.json
par

In [84]:
import pickle

path = "/d/shared/users/jag/penfed/autoindirectv1/nationalmodel3test/supporting_model/model.obj"

with open(path, "rb") as f:
    model = pickle.load(f)


In [85]:
type(model)

dict

In [86]:
model.keys()

dict_keys(['zaml_ver', 'dep_vers'])

In [75]:
import pickle

path = "/d/shared/users/jag/penfed/autoindirectv1/nationalmodel3test/supporting_model/pipeline.obj"

with open(path, "rb") as f:
    pipeline = pickle.load(f)


In [79]:
pipeline.keys()

dict_keys(['zaml_ver', 'dep_vers'])

In [80]:
pipeline['dep_vers']

{'dill': '0.3.6',
 'docmill': '0.1.2',
 'feather-format': '0.4.1',
 'graphviz': '0.21',
 'h5py': '3.15.1',
 'hyperopt': '0.2.7',
 'ipython': '8.30.0',
 'ipywidgets': '8.1.8',
 'joblib': '1.5.3',
 'jupyter-book': '2.1.0',
 'jupyter_server_mathjax': '0.2.6',
 'kaleido': '1.2.0',
 'lightgbm': '3.3.5',
 'matplotlib': '3.10.8',
 'numba': '0.56.4',
 'numpy': '1.23.5',
 'openpyxl': '3.1.5',
 'pandas': '2.0.2',
 'pg8000': '1.31.5',
 'plotly': '5.20.0',
 'pyarrow': '11.0.0',
 'pyarrow_hotfix': '0.7',
 'PypeR': '1.1.2',
 'PyYAML': '6.0.3',
 'schedule': '1.2.2',
 'scikit-learn': '1.2.2',
 'scipy': '1.10.1',
 'seaborn': '0.13.2',
 'shap': '0.41.0',
 'simplejson': '3.20.2',
 'Sphinx': '8.0.2',
 'sphinx-rtd-theme': '3.1.0',
 'sphinxcontrib-applehelp': '1.0.7',
 'sphinxcontrib-bibtex': '2.6.1',
 'sphinxcontrib-devhelp': '1.0.5',
 'sphinxcontrib-htmlhelp': '2.0.4',
 'sphinxcontrib-qthelp': '1.0.6',
 'sphinxcontrib-serializinghtml': '1.1.9',
 'sqlalchemy': '1.4.54',
 'sqlalchemy_utils': '0.42.1',
 'sta

In [82]:
asset = json.load(open('/d/shared/users/jag/penfed/autoindirectv1/nationalmodel3test/supporting_model/asset.json', 'r'))

In [83]:
asset

{'data': {'inq': {'asset': 'equifax/cms_6/inquiry.json',
   'data': '#/d/shared/users/jag/penfed/autoindirectv1/nationalmodel3test/lookalike_sample/inq_sampled_files.json',
   'io_params': {'drop_duplicates': True,
    'keep_keys': '/d/shared/users/jag/penfed/autoindirectv1/nationalmodel3test/lookalike_sample/keep_keys.json',
    'keep_features': ['inq_inquiries__sum'],
    'memory_efficient': True}},
  'app': {'asset': {'info': {'key': 'ZEST_KEY', 'app_date': 'appDate'},
    'table_name': 'app',
    'table_type': 'one_to_one',
    'feature_engineering': 'one_to_one'},
   'data': '#/d/shared/users/jag/penfed/autoindirectv1/nationalmodel3test/lookalike_sample/app_sampled_files.json',
   'io_params': {'drop_duplicates': True,
    'keep_keys': '/d/shared/users/jag/penfed/autoindirectv1/nationalmodel3test/lookalike_sample/keep_keys.json',
    'keep_features': ['appDate',
     'is_client',
     'bisgWhite',
     'bisgBlack',
     'bisgAian',
     'bisgApi',
     'bisgHispanic',
     'bisgMu

In [72]:
description = json.load(open('/d/shared/users/jag/penfed/autoindirectv1/nationalmodel3test/supporting_model/data_description.json', 'r'))

In [73]:
description

{'baseline_model': {'data_split': {'train': {'start_date': '2016-04-01',
    'end_date': '2016-05-01'},
   'valid': {'start_date': '2016-05-01', 'end_date': '2016-06-01'},
   'test': {'start_date': '2016-06-01', 'end_date': '2016-08-01'}},
  'target': 'final_DQ60_m24',
  'data_sources': ['inq', 'app', 'target'],
  'data_summary': {'train': {'num_of_rows': {'train_app': 736833}},
   'valid': {'num_of_rows': {'valid_app': 747280}},
   'test': {'num_of_rows': {'test_app': 1491337}}},
  'alias': 'national_model'},
 'clients': {'clients': None,
  'data_split': None,
  'target': None,
  'data_summary': None}}

## We now have our model and we can train the full on with [_build_reduced_split_model](https://github.com/Katlean/model-engine/blob/c78ae3f22054dbc9b08b4c0540318ee0976ec889/model_engine/power/post_sale/national_model_builder.py#L318)
```python
self._build_reduced_split_model(self.artifacts_path, supporting_model_path, self.manifest)
```

```python
def _build_reduced_split_model(self, model_output_path, full_data_split_model_path, manifest):
    mb_asset = global_load_data(os.path.join(full_data_split_model_path, 'asset.json'))
    adapted_mb_asset = self.make_fold_valid_asset(mb_asset, full_data_split_model_path)
    build_model(adapted_mb_asset, model_output_path, manifest=manifest)
    # Add both data_description and unfold_data_description
    self._add_data_description(model_output_path, model_output_path, 'data_description.json')
        self._add_data_description(os.path.join(model_output_path, 'supporting_model'), model_output_path, 'unfold_data_description.json')
```

In [141]:
nmb.manifest

{'data': {'train_app': {'type': 'dataframe',
   'path': 'app/split=train/data=national/data_0.parquet',
   'required': False},
  'valid_app': {'type': 'dataframe',
   'path': 'app/split=valid/data=national/data_0.parquet',
   'required': False},
  'test_app': {'type': 'dataframe',
   'path': 'app/split=test/data=national/data_0.parquet',
   'required': False},
  'train_target': {'type': 'series',
   'path': 'target/split=train/data=national/data_0.parquet',
   'required': False},
  'valid_target': {'type': 'series',
   'path': 'target/split=valid/data=national/data_0.parquet',
   'required': False},
  'test_target': {'type': 'series',
   'path': 'target/split=test/data=national/data_0.parquet',
   'required': False},
  'train_fe_data': {'type': 'dataframe',
   'path': 'fe_data/split=train/data=national/data_0.parquet',
   'required': False},
  'valid_fe_data': {'type': 'dataframe',
   'path': 'fe_data/split=valid/data=national/data_0.parquet',
   'required': False},
  'test_fe_data': {

In [142]:
nmb.artifacts_path

'/d/shared/users/jag/penfed/autoindirectv1/nationalmodel3test'

# load mb_asset



In [144]:
from model_engine.io.utils import global_get_bucket_prefix, global_load_data

mb_asset = global_load_data(os.path.join(supporting_model_path, 'asset.json'))

In [148]:
mb_asset['config']

{'data_split': {'train': {'start_date': '2016-04-01',
   'end_date': '2016-05-01'},
  'valid': {'start_date': '2016-05-01', 'end_date': '2016-06-01'},
  'test': {'start_date': '2016-06-01', 'end_date': '2016-08-01'}},
 'target': 'final_DQ60_m24',
 'base_features': '/d/shared/users/jag/penfed/autoindirectv1/test_only_5_features/keep_features_national_model.json',
 'memory_efficient': True,
 'pipeline_factory': 'BureauPipelineFactory',
 'bivariate_fe_instructions': [],
 'monotonic_constraints_list': [],
 'exclusion_list': [],
 'feature_definition_list': [],
 'key_factor_mapping_list': [],
 'fold_valid': True,
 'mandatory_features': []}

In [150]:
nmb.artifacts_path

'/d/shared/users/jag/penfed/autoindirectv1/nationalmodel3test'

In [151]:
mb_asset['config']['pipeline_factory']

'BureauPipelineFactory'

In [145]:
adapted_mb_asset = nmb.make_fold_valid_asset(mb_asset, supporting_model_path)

```python
def make_fold_valid_asset(self, original_mb_asset, full_split_model_path):
        '''
        This method adapts the existing model builder asset to accommodate the "folding validation into training" case.
        '''
        # deep copy to avoid overwriting existing mb_asset
        mb_asset = copy.deepcopy(original_mb_asset)
        # extract the best number of iterations from the previous model
        best_num_estimator = self._get_best_num_iteration(full_split_model_path)
        # adapt the data split section by merging validation into training set
        if mb_asset['config']['data_split']['train']['end_date'] != mb_asset['config']['data_split']['valid']['start_date']:
            raise NationalModelBuilderError(f'The training and validation periods have a gap, which prohbits the automatic fold-validation functionality.')
        mb_asset['config']['data_split']['train']['end_date'] = mb_asset['config']['data_split']['valid']['end_date']
        mb_asset['config']['data_split'].pop('valid', None)
        # adapt the pipeline factory
        # pipeline factory might be a string that denotes pipeline name or a nested dict that contains the detailed pipeline specification
        if isinstance(mb_asset['config']['pipeline_factory'], str):
            all_pipelines = get_default_factory_configs()
            pipeline_spec = all_pipelines[mb_asset['config']['pipeline_factory']]
        else:
            pipeline_spec = mb_asset['config']['pipeline_factory']
        pipeline_spec['model']['params'].pop('early_stopping_rounds', None)
        pipeline_spec['model']['params']['n_estimators'] = best_num_estimator
        mb_asset['config']['pipeline_factory'] = pipeline_spec
        return mb_asset
```

## We can see that [make_fold_valid_asset](https://github.com/Katlean/model-engine/blob/c78ae3f22054dbc9b08b4c0540318ee0976ec889/model_engine/power/post_sale/national_model_builder.py#L199) updates our train timeframe to include valid and we grab the pipeline factor but change the estimators from the best_num_estimator which we grabb with [_get_best_num_iteration](https://github.com/Katlean/model-engine/blob/c78ae3f22054dbc9b08b4c0540318ee0976ec889/model_engine/power/post_sale/national_model_builder.py#L466) which we called our on our split model path

```python
def _get_best_num_iteration(self, model_path):
    '''
    This method extracts the best number of iterations from existing model, which should be called only in the case when config:model_config:fold_valid=True.
    '''
    if global_path_exists(os.path.join(model_path, 'best_model_params.json')):
        best_model_params = global_load_data(os.path.join(model_path, 'best_model_params.json'))
        return best_model_params['best_ntree_limit']
    else:
        raise NationalModelBuilderError(f'{model_path} does not contain the best_model_params.json file.')
```

In [152]:
supporting_model_path

'/d/shared/users/jag/penfed/autoindirectv1/nationalmodel3test/supporting_model'

In [154]:
best_model_params = json.load(open('/d/shared/users/jag/penfed/autoindirectv1/nationalmodel3test/supporting_model/best_model_params.json', 'rb'))

In [155]:
best_model_params

{'best_score': 0.6268342834597741, 'best_iteration': 9, 'best_ntree_limit': 10}

In [149]:
adapted_mb_asset['config']

{'data_split': {'train': {'start_date': '2016-04-01',
   'end_date': '2016-06-01'},
  'test': {'start_date': '2016-06-01', 'end_date': '2016-08-01'}},
 'target': 'final_DQ60_m24',
 'base_features': '/d/shared/users/jag/penfed/autoindirectv1/test_only_5_features/keep_features_national_model.json',
 'memory_efficient': True,
 'pipeline_factory': {'transformers': [{'zaml_class': 'FillNA',
    'params': {'replace_by': -1, 'add_flags': False}},
   {'zaml_class': 'GeneralFeatureSelection',
    'params': {'features': ['app_is_client']}}],
  'model': {'zaml_class': 'XGBoostModel',
   'params': {'n_estimators': 10,
    'learning_rate': 0.02,
    'max_depth': 6,
    'subsample': 0.9,
    'scale_pos_weight': 4,
    'colsample_bytree': 0.25,
    'min_child_weight': 150,
    'seed': 12,
    'reg_lambda': 5,
    'eval_metric': 'auc',
    'verbose_eval': 100,
    'enable_index_matching': True,
    'tree_method': 'hist',
    'n_min_estimators': 0,
    'booster': 'gbtree',
    'gamma': 0,
    'max_delt

# we then call build model again with the adapted mb_asset 

In [156]:
state_second_model = asset_parser(adapted_mb_asset)


/home/jag/.conda/envs/demoanalysis/lib/python3.10/site-packages/fsspec/registry.py:301: UserWarning: Your installed version of s3fs is very old and known to cause
severe performance issues, see also https://github.com/dask/dask/issues/10276

To fix, you should specify a lower version bound on s3fs, or
update the current installation.

  warnings.warn(s3_msg)
/home/jag/.conda/envs/demoanalysis/lib/python3.10/site-packages/fsspec/registry.py:301: UserWarning: Your installed version of s3fs is very old and known to cause
severe performance issues, see also https://github.com/dask/dask/issues/10276

To fix, you should specify a lower version bound on s3fs, or
update the current installation.

  warnings.warn(s3_msg)
/home/jag/.conda/envs/demoanalysis/lib/python3.10/site-packages/fsspec/registry.py:301: UserWarning: Your installed version of s3fs is very old and known to cause
severe performance issues, see also https://github.com/dask/dask/issues/10276

To fix, you should specify a lower 

In [158]:
# Initialize ModelBuilder for second model
builder_second_model = ModelBuilder(
    data=state_second_model['data'], 
    data_split=state_second_model['config']['data_split'], 
    archive_mode='regular'
)

# Configure the builder to return outputs in memory
builder_second_model.configure_run(return_outputs=True)

# Build the model
print('Building second model...')
artifact_output_second_model = builder_second_model.generate(
    input_data=state_second_model['data'], 
    input_asset=state_second_model['input_asset'], 
    **state_second_model['config']
)

print('Second model build complete!')

INFO:zaml.artifact_engine.logger:Executing InputArtifact <input_asset>...
INFO:zaml.artifact_engine.logger:Executing InputArtifact <input_data>...
INFO:zaml.artifact_engine.logger:Executing InputArtifact <monotonic_constraints_list>...
INFO:zaml.artifact_engine.logger:Executing InputArtifact <data_split>...
INFO:zaml.artifact_engine.logger:Executing InputArtifact <train_sample_weight>...
INFO:zaml.artifact_engine.logger:Not all required inputs are available for optional artifact <train_sample_weight>, thus it will be omitted.
INFO:zaml.artifact_engine.logger:Executing InputArtifact <test_sample_weight>...
INFO:zaml.artifact_engine.logger:Not all required inputs are available for optional artifact <test_sample_weight>, thus it will be omitted.
INFO:zaml.artifact_engine.logger:Executing InputArtifact <pipeline_factory>...
INFO:zaml.artifact_engine.logger:Executing InputArtifact <exclusion_list>...
INFO:zaml.artifact_engine.logger:Executing InputArtifact <bivariate_fe_instructions>...
INF

Building second model...


INFO:zaml.artifact_engine.logger:Finished <parsed_monotonic_constraints_list>, total time spent: 0:00:00.533819
INFO:zaml.artifact_engine.logger:Executing SplitterArtifact <splitter>...
INFO:zaml.artifact_engine.logger:Finished <splitter>, total time spent: 0:00:00.000737
INFO:zaml.artifact_engine.logger:Executing SeriesArtifact <target>...
INFO:zaml.artifact_engine.logger:Finished <target>, total time spent: 0:00:00.007226
INFO:zaml.artifact_engine.logger:Executing AppArtifact <app>...
INFO:zaml.artifact_engine.logger:Finished <app>, total time spent: 0:00:00.011907
INFO:zaml.artifact_engine.logger:Executing DataArtifact <data>...
INFO:zaml.artifact_engine.logger:Finished <data>, total time spent: 0:00:00.000553
INFO:zaml.artifact_engine.logger:Executing PipeFactoryConfigArtifact <pipe_factory_config>...
INFO:zaml.artifact_engine.logger:Finished <pipe_factory_config>, total time spent: 0:00:00.000705
INFO:zaml.artifact_engine.logger:Executing ExclusionListParser <parsed_exclusion_list

-------------------------
Name: app
Transformer type: None
Number of features: 18
Time spent: 0.073s
-------------------------
Name: inq
Transformer type: None
Number of features: 1
Time spent: 0.021s
-------------------------
Name: app FE
Transformer type: OneToOneEngine
Number of features: 0
Time spent: 0.076s
-------------------------
Name: inq FE
Transformer type: InquiryFeatureEngine
Number of features: 1
Time spent: 0.041s


-------------------------
Name: Merge data
Transformer type: Concat
Number of features: 1
Time spent: 3.324s
-------------------------
Name: FillNA
Transformer type: FillNA
Number of features: 1
Time spent: 0.024s
-------------------------
Name: GeneralFeatureSelection
Transformer type: GeneralFeatureSelection
Number of features: 1
Time spent: 0.002s


...

-------------------------
Name: Merge data
Transformer type: Concat
Number of features: 1
Time spent: 3.087s
-------------------------
Name: FillNA
Transformer type: FillNA
Number of features: 1
Time spent: 0.018s
-------------------------
Name: GeneralFeatureSelection
Transformer type: GeneralFeatureSelection
Number of features: 1
Time spent: 0.001s
-------------------------
Name: app
Transformer type: None
Number of features: 18
Time spent: 0.000s
-------------------------
Name: inq
Transformer type: None
Number of features: 1
Time spent: 0.000s
-------------------------
Name: app FE
Transformer type: OneToOneEngine
Number of features: 0
Time spent: 0.058s
-------------------------
Name: inq FE
Transformer type: InquiryFeatureEngine
Number of features: 1
Time spent: 0.023s



INFO:zaml.artifact_engine.logger:Finished <test_fe_data>, total time spent: 0:00:03.511672
INFO:zaml.artifact_engine.logger:Executing StaticAssetArtifact <static_asset>...
INFO:zaml.artifact_engine.logger:Finished <static_asset>, total time spent: 0:00:00.034289
INFO:zaml.artifact_engine.logger:Executing TrainHistoryArtifact <train_history>...
INFO:zaml.artifact_engine.logger:Finished <train_history>, total time spent: 0:00:00.000376
INFO:zaml.artifact_engine.logger:Executing BestModelParamsArtifact <best_model_params>...
INFO:zaml.artifact_engine.logger:Finished <best_model_params>, total time spent: 0:00:00.000551
INFO:zaml.artifact_engine.logger:Executing ScoresArtifact <train_scores>...
INFO:zaml.artifact_engine.logger:Finished <train_scores>, total time spent: 0:00:00.069764
INFO:zaml.artifact_engine.logger:Executing CalibrationObjectArtifact <calibration_object>...


-------------------------
Name: Merge data
Transformer type: Concat
Number of features: 1
Time spent: 3.396s
-------------------------
Name: FillNA
Transformer type: FillNA
Number of features: 1
Time spent: 0.019s
-------------------------
Name: GeneralFeatureSelection
Transformer type: GeneralFeatureSelection
Number of features: 1
Time spent: 0.001s


INFO:zaml.artifact_engine.logger:Finished <calibration_object>, total time spent: 0:00:49.537273
INFO:zaml.artifact_engine.logger:Executing SubmodelScoresArtifact <train_submodel_scores>...
INFO:zaml.artifact_engine.logger:Finished <train_submodel_scores>, total time spent: 0:00:00.000570
INFO:zaml.artifact_engine.logger:Executing ScoresArtifact <test_scores>...
INFO:zaml.artifact_engine.logger:Finished <test_scores>, total time spent: 0:00:00.073249
INFO:zaml.artifact_engine.logger:Executing SubmodelScoresArtifact <test_submodel_scores>...
INFO:zaml.artifact_engine.logger:Finished <test_submodel_scores>, total time spent: 0:00:00.000453
INFO:zaml.artifact_engine.logger:Executing FeatureDefinition <feature_definition>...
INFO:zaml.artifact_engine.logger:Finished <feature_definition>, total time spent: 0:00:00.010329
INFO:zaml.artifact_engine.logger:Executing KeyFactorMapping <key_factors_mapping>...
INFO:zaml.artifact_engine.logger:Finished <key_factors_mapping>, total time spent: 0:00

Second model build complete!


In [159]:
handler_second_model = FileSystemHandler()
prefix = nmb.artifacts_path

In [160]:
data_archiver_object_second_model = DataArchiver(
    output_dir=prefix, handler=handler_second_model, manifest=nmb.manifest, save_manifest=True, extras='allow'
)
data_archiver_object_second_model.archive(artifact_output_second_model)

In [166]:
artifact_output_second_model.keys()

dict_keys(['asset', 'versions', 'parsed_monotonic_constraints_list', 'splitter', 'train_target', 'train_app', 'test_target', 'test_app', 'train_data_summary', 'test_data_summary', 'pipeline', 'fit_time_info', 'model', 'train_fe_data', 'test_fe_data', 'static_asset', 'train_history', 'best_model_params', 'train_scores', 'calibration_object', 'test_scores', 'feature_definition', 'key_factors_mapping', 'value_based_key_factor_mapping', 'feature_importance', 'model_strategy', 'mrm_pipeline', 'score_recalibration_mapping', 'train_auc', 'train_ks', 'test_auc', 'test_ks', 'top_features', 'keep_features', 'train_zest_scores', 'test_zest_scores'])

In [173]:
full_artifact['pipeline'].model.__dict__

{'_zaml_ver': '34.4.1',
 '_other_params': {'eval_metric': 'auc', 'tree_method': 'hist'},
 'model_name': 'xgboost_model_ffaadb60-e65e-4637-a145-b80f814cda71',
 'model': <xgboost.core.Booster at 0x7f65b23cfdf0>,
 'feature_types': [('inq_inquiries__sum', 'float32')],
 'enable_index_matching': True,
 'explainer_type': 'TreeExplainer',
 'max_depth': 6,
 'learning_rate': 0.02,
 'n_estimators': 25000,
 'early_stopping_rounds': 200,
 'n_min_estimators': 0,
 'objective': 'binary:logistic',
 'booster': 'gbtree',
 'gamma': 0,
 'min_child_weight': 150,
 'max_delta_step': 0,
 'subsample': 0.9,
 'colsample_bytree': 0.25,
 'colsample_bylevel': 1,
 'reg_alpha': 0,
 'reg_lambda': 5,
 'scale_pos_weight': 4,
 'max_leaves': 0,
 'base_score': 0.5,
 'seed': 12,
 'monotone_constraints': {'trade_months_since_openDate__max': -1,
  'trade_months_since_openDate__max_by_acctType_auto': -1,
  'trade_months_since_openDate__max_by_acctType_charge_card': -1,
  'trade_months_since_openDate__max_by_acctType_credit_card

In [175]:
full_artifact['pipeline'].model.n_estimators

25000

In [176]:
artifact_output_second_model['pipeline'].model.n_estimators

10